In [8]:
# import dependencies

import pandas as pd
import numpy as np
import math
import sympy as sp
import os
import locale
from copy import deepcopy
import scipy.stats as stats
import matplotlib as mpl
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
from matplotlib import cm
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from scipy.optimize import curve_fit
import statsmodels.api as sm
import statsmodels.stats as smstats
import scipy.stats as stats
from sklearn.linear_model import LinearRegression

np.seterr(all='raise')

pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

locale.setlocale(locale.LC_NUMERIC, "de_DE")
locale._override_localeconv["frac_digits"] = 2
plt.style.use('default')

pgf_with_latex = {                      # setup matplotlib to use latex for output
    "pgf.texsystem": "pdflatex",        # change this if using xetex or lautex
    "text.usetex": False,                # use LaTeX to write all text
    "font.family": "Times New Roman",
    "font.serif": [],                   # blank entries should cause plots 
    "font.sans-serif": [],              # to inherit fonts from the document
    "font.monospace": [],
    "axes.labelsize": 16,               # LaTeX default is 10pt font.
    "font.size": 16,
    'mathtext.fontset': 'stix',
    "legend.fontsize": 16,               # Make the legend/label fonts 
    "xtick.labelsize": 16,               # a little smaller
    "ytick.labelsize": 16,
    "pgf.preamble": "\n".join([ # plots will use this preamble
        r"\usepackage[utf8]{inputenc}",
        r"\usepackage[T1]{fontenc}",
        r"\usepackage[detect-all,locale=DE]{siunitx}",
        ]),
    "axes.formatter.use_locale": True,
    }

mpl.rcParams.update(pgf_with_latex)

pointstyle1_dict = {'s':20, 'facecolor': 'None', 'edgecolors': 'black'}
pointstyle2_dict = {'s':20, 'facecolor': 'None', 'edgecolors': 'blue'}
linestyle1_dict = {'linestyle': 'solid', 'linewidth': 1}
linestyle2_dict = {'linestyle': 'dashed', 'linewidth': 1}

In [9]:
# data imports

casename = '12-11-23'
caseloc = f'C:\\Users\\LENOVO\\Documents\\TA\\Data\\{casename}'
irr_ = 500; v0_ = 1e-2; T0_ = 300
offset_time_ = 240

os.chdir(f'{caseloc}\\tabel')
df_aktual = pd.read_csv(f'hasil_{casename}.csv')
df_aktual['case'] = df_aktual['case'].astype('int')
df_aktual['irr'] = df_aktual['irr'].astype('int')

delta_t = df_aktual.loc[:, 'times'].values[1] - df_aktual.loc[:, 'times'].values[0]


In [10]:
# forward ventilation calcs
V_tot = 0.15 * 3**2 + 3.5
V_kamar = 3**2 * 2.5

def calc_rt(df_this):
    Vdot_ = df_this['v_stream'].values * df_this['A'].values[0]
    rt_ = [(V_tot / Vdot_[0]) + (1 - (V_tot / Vdot_[0])) * math.exp(-Vdot_[0] * delta_t / V_kamar)]
    for it1, it2 in enumerate(Vdot_[1:]):
        rt_.append((V_tot / it2) + (rt_[-1] - (V_tot / it2)) * math.exp( -it2 * delta_t / V_kamar))
    return rt_

In [11]:
# Label line with line2D label data
def labelLine(line,x,label=None,align=True,**kwargs):

    ax = line.axes
    xdata = line.get_xdata()
    ydata = line.get_ydata()

    if (x < xdata[0]) or (x > xdata[-1]):
        print('x label location is outside data range!')
        return

    #Find corresponding y co-ordinate and angle of the line
    ip = 1
    for i in range(len(xdata)):
        if x < xdata[i]:
            ip = i
            break

    y = ydata[ip-1] + (ydata[ip]-ydata[ip-1])*(x-xdata[ip-1])/(xdata[ip]-xdata[ip-1])

    if not label:
        label = line.get_label()

    if align:
        #Compute the slope
        dx = xdata[ip] - xdata[ip-1]
        dy = ydata[ip] - ydata[ip-1]
        ang = math.degrees(math.atan2(dy,dx))

        #Transform to screen co-ordinates
        pt = np.array([x,y]).reshape((1,2))
        trans_angle = ax.transData.transform_angles(np.array((ang,)),pt)[0]

    else:
        trans_angle = 0

    #Set a bunch of keyword arguments
    if 'color' not in kwargs:
        kwargs['color'] = line.get_color()

    if ('horizontalalignment' not in kwargs) and ('ha' not in kwargs):
        kwargs['ha'] = 'center'

    if ('verticalalignment' not in kwargs) and ('va' not in kwargs):
        kwargs['va'] = 'center'

    if 'backgroundcolor' not in kwargs:
        kwargs['backgroundcolor'] = ax.get_facecolor()

    if 'clip_on' not in kwargs:
        kwargs['clip_on'] = True

    if 'zorder' not in kwargs:
        kwargs['zorder'] = 2.5

    ax.text(x,y,label,rotation=trans_angle,**kwargs)

def labelLines(lines,align=True,xvals=None,**kwargs):

    ax = lines[0].axes
    labLines = []
    labels = []

    #Take only the lines which have labels other than the default ones
    for line in lines:
        label = line.get_label()
        if "_line" not in label:
            labLines.append(line)
            labels.append(label)

    if xvals is None:
        xmin,xmax = ax.get_xlim()
        xvals = np.linspace(xmin,xmax,len(labLines)+2)[1:-1]

    for line,x,label in zip(labLines,xvals,labels):
        labelLine(line,x,label,align,**kwargs)

In [5]:
# smooth responses
def polynom_curve(x, a, b, c):
    return a*pow(x,2) + b*x + c

def smooth_respons(casename, caseloc, df_aktual, offset_time_):
    df_train = deepcopy(df_aktual); df_train = df_train[df_train['times'] > offset_time_]
    df_train['v_fluid'] = df_train['v_fluid'] - v0_
    df_train['v_stream'] = df_train['v_stream'] - v0_
    df_train['T_fluid'] = df_train['T_fluid'] - T0_
    df_train['T_stream'] = df_train['T_stream'] - T0_
    df_train['T_glass'] = df_train['T_glass'] - T0_
    df_train['T_abs'] = df_train['T_abs'] - T0_
    df_train['T_insul'] = df_train['T_insul'] - T0_
    
    col_smooth = {'v_fluid': [], 'v_stream': [], 'T_fluid': [], 'T_stream': [], 'T_glass': [], 'T_abs': [], 'T_insul': [],
                  'Pgrady_stream': [], 'vgradz_fluid': [], 'Tgradz_fluid': []}
    
    df_smooth = pd.DataFrame(columns=['kasus', 'variabel', 'a', 'b', 'c'])
    df_smooth_data = pd.DataFrame(columns=list(col_smooth.keys()))
    
    all_time = []
    
    for case_name, df_case in df_train.groupby('case'):
        time_ = df_aktual[df_aktual['case'] == case_name]['times'].values
        all_time.extend(time_)
        for vars in list(col_smooth.keys()):
            popt, pcov = curve_fit(polynom_curve, df_case['times'].values, df_case[vars].values)
            df_smooth.loc[df_smooth.shape[0],:] = [case_name, vars, *popt]
            col_smooth[vars].extend([polynom_curve(x, *popt) for x in time_])
    
    df_smooth_data['case'] = df_aktual['case'].values
    df_smooth_data['times'] = all_time
    for vars, vars_curve in col_smooth.items():
        if any([vars[:2] == 'T_', vars in ['Tgradz_fluid']]) is True:
            df_smooth_data[vars] = df_aktual[vars]
        else:
            df_smooth_data[vars] = np.array(vars_curve)
    
    df_smooth_data['v_fluid'] = df_smooth_data['v_fluid'] + v0_
    df_smooth_data['v_stream'] = df_smooth_data['v_stream'] + v0_
    # df_smooth_data['T_fluid'] = df_smooth_data['T_fluid'] + T0_
    # df_smooth_data['T_stream'] = df_smooth_data['T_stream'] + T0_
    # df_smooth_data['T_glass'] = df_smooth_data['T_glass'] + T0s_
    # df_smooth_data['T_abs'] = df_smooth_data['T_abs'] + T0_
    # df_smooth_data['T_insul'] = df_smooth_data['T_insul'] + T0_
    
    for it1 in ['u', 'v', 'w', 'T', 'k', 'omega']:
        df_smooth_data[f'rmsr_{it1}'] = df_aktual[f'rmsr_{it1}']
        df_smooth_data[f'cfl_{it1}'] = df_aktual[f'cfl_{it1}']
    
    df_smooth.to_csv(f'{caseloc}\\tabel\\smooth_coef_{casename}.csv')
    print(f'Saved to smooth_coef_{casename}.csv')    
    
    return df_smooth_data

df_ = smooth_respons(casename, caseloc, df_aktual, offset_time_)
df_.to_csv(f'{caseloc}\\tabel\\smooth_{casename}.csv')
print(f'Saved to smooth_{casename}.csv')

c:\Users\LENOVO\Documents\TA\Repo\.venv\Lib\site-packages\scipy\optimize\_minpack_py.py:906: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',


Saved to smooth_coef_12-11-23.csv
Saved to smooth_12-11-23.csv


In [6]:
# parametric design and values
# df_ = deepcopy(df_aktual)

mean_H = 0.575; range_H = 0.425
mean_W = 1.75; range_W = 0.75

channel_H_transform = lambda x: mean_H + (x/1.414) * range_H
channel_W_transform = lambda x: mean_W + (x/1.414) * range_W

channel_H = np.array([0.27, 0.27, 0.88, 0.88, 1, 0.15, 0.575, 0.575, 0.575])
channel_W = np.array([1.22, 2.28, 1.22, 2.28, 1.75, 1.75, 2.5, 1, 1.75])
channel_A = channel_H * channel_W
channel_HW = channel_H / channel_W
channel_H2 = np.array([x**2 for x in channel_H])
channel_W2 = np.array([x**2 for x in channel_W])

df_['W'] = np.array([channel_W[x-1] for x in df_['case'].values])
df_['H'] = np.array([channel_H[x-1] for x in df_['case'].values])
df_['A'] = np.array([channel_A[x-1] for x in df_['case'].values])
df_['HW'] = np.array([channel_HW[x-1] for x in df_['case'].values])

# calc values and characteristics
# Dh oriented

irr_glass_ = 0.06 * irr_; irr_abs_ = 0.89 * 0.95 * irr_
eps_glass = 0.89; eps_abs = 0.95
k_gl = 0.9; k_abs = 0.83; k_ins = 0.159
rho_inf = 1.1614; rho_gl = 2400; rho_abs = 413; rho_ins = 1050
cp_gl = 840; cp_abs = 1200; cp_ins = 1100
Tsky_ = 0.0552 * pow(300, 1.5)
hins_ = 10

htf_area_l = df_['W'] * 2.5
V_fluid_l = df_['A'] * 2.5
V_g_abs_l = df_['W'] * 0.025 * 2.5
V_ins_l = df_['W'] * 0.05 * 2.5

rho_fl = lambda x: 1.1614 - 0.00353 * (x - 300)
mu_fl = lambda x: (1.846 + 0.00472 * (x - 300)) * pow(10, -5)
k_fl = lambda x: 0.0263 + 0.000074 * (x - 300)
cp_fl = lambda x: (1.007 + 0.00004 * (x - 300)) * pow(10, 3)
nu_ = 1.48 * pow(10, -5)

rho_fl_ = np.array([rho_fl(x) for x in df_['T_stream']])
mu_fl_ = np.array([mu_fl(x) for x in df_['T_stream']])
k_fl_ = np.array([k_fl(x) for x in df_['T_stream']])
cp_fl_ = np.array([cp_fl(x) for x in df_['T_stream']])

Dh_ = 2 * df_['H'] * df_['W'] / (df_['H'] + df_['W'])

df_['Dh'] = Dh_

# stream summaries
df_['T_surface'] = (df_['T_glass'] + df_['T_abs']) / 2

df_['Re'] = rho_fl_ * df_['v_stream'] * Dh_ / mu_fl_
df_['Gr'] = 9.81 * np.array([np.max([x,0]) for x in df_['T_surface'] - df_['T_stream']]) * np.array([x**3 for x in Dh_]) / (df_['T_stream'] * nu_**2)
# df_['Gr'] = 9.81 * df_['Tgradz_fluid'].values * (df_['T_abs'] - df_['T_stream']) * np.array([x**4 for x in Dh_]) / (df_['T_stream'] * nu_**2)
df_['Ra'] = df_['Gr'] * 0.71
df_['Cf'] = mu_fl_ * df_['vgradz_fluid'] / (rho_fl_ * np.array([x**2 for x in df_['v_stream'].values]))
df_['Nu'] = df_['Tgradz_fluid'].values * Dh_ / (df_['T_abs'].values - df_['T_stream'].values) # kali dua karena dist Dh / 2 tp lupa dimasukin ke cfd_analysis4.py (skrg udh ntar diilangin kalo run lagi)
df_['Nu_eta'] = df_['Nu']
df_['Nu_psi'] = df_['Nu']
df_['h'] = df_['Nu'] * k_fl_ / Dh_
df_['h_eta'] = df_['h']
df_['h_psi'] = df_['h']

df_['mdot'] = rho_fl_ * df_['A'] * df_['v_stream']
df_['f'] = np.array([np.abs(x) for x in -df_['Pgrady_stream'] * 2.5 / (rho_fl_ * np.array([y**2 for y in df_['v_stream']]) / 2)])
df_['Cd'] = np.full(shape=(df_.shape[0],), fill_value=0.57) # df_['mdot'] / (df_['A'] * np.array([np.sqrt(2 * rho_fl(x) * df_['Pgrady_stream'].values[x]) for x in list(df_.index)]))
df_['Fo'] = np.array([k_fl(x) / (rho_fl(x) * cp_fl(x)) for x in df_['T_fluid'].values]) * df_['times'].values

df_['NuFo'] = df_['Nu'] * df_['Fo']
df_['GrCf'] = np.array([pow(x, 0.25) * y / 2 for x,y in zip(df_['Gr'].values, df_['Cf'].values)])
df_['q'] = irr_ * df_['W'].values * 2.5

df_['q_stream'] = np.array([channel_A[x-1] for x in df_['case'].values]) * rho_fl_ * 2.5 * cp_fl_ * df_['T_fluid']
df_['q_g'] = np.array([channel_A[x-1] for x in df_['case'].values]) * rho_gl * 2.5 * cp_gl * df_['T_glass']
df_['q_abs'] = np.array([channel_A[x-1] for x in df_['case'].values]) * rho_abs * 2.5 * cp_abs * df_['T_abs']
df_['q_ins'] = np.array([channel_A[x-1] for x in df_['case'].values]) * rho_ins * 2.5 * cp_ins * df_['T_insul']

# calc validation

def calc_validation(df_data):
    err_g = []; err_fl = []; err_abs = []; err_ins = []
    err_type2 = []
    
    for case_name, df_case in df_data.groupby('case'):
        time_ = df_case['times'].values
        
        A_ = np.zeros(shape=(4,4), dtype = float)
        B_ = np.zeros(shape=(1,4), dtype = float)[0]
        X_ = deepcopy(B_)
        
        delta_t = (time_[1] - time_[0]) / 2
        htf_area = df_case['W'].values[0] * 2.5
        V_fluid = df_case['A'].values[0] * 2.5
        V_g_abs = df_case['W'].values[0] * 0.025 * 2.5
        V_ins = df_case['W'].values[0] * 0.05 * 2.5
        
        T_glass_prev = 300; T_fluid_prev = 300; T_abs_prev = 300; T_ins_prev = 300
        
        for it1, it2 in enumerate(time_):
            T_fluid_l = df_case['T_fluid'].values[it1]
            T_stream_l = df_case['T_stream'].values[it1]
            T_glass_l = df_case['T_glass'].values[it1]
            T_abs_l = df_case['T_abs'].values[it1]
            T_ins_l = df_case['T_insul'].values[it1]
            h_l = df_case['h'].values[it1]
            
            k_fl_l = k_fl(T_fluid_l)
            rho_fl_l = rho_fl(T_fluid_l)
            rho_stream_l = rho_fl(T_stream_l)
            cp_fl_l = cp_fl(T_fluid_l)
            
            # energy balance
            hsky_ = 5.67 * pow(10, -8) * 0.89 * (T_glass_l + Tsky_) * (T_glass_l**2 + Tsky_**2) * \
                (T_glass_l - Tsky_) / (T_glass_l - 300)
            hrwg_ = 5.67 * pow(10, -8) * (T_abs_l**2 + T_glass_l**2) * (T_abs_l + T_glass_l) \
                / ((1/eps_glass) + (1/eps_abs) - 1)
            
            U_gfl = 1 / ((1 / h_l) + (0.025 / (2 * k_gl)) + (df_case['H'].values[0] / (2 * k_fl_l)))
            U_absfl = 1 / ((1 / h_l) + (0.025 / (2 * k_abs)) + (df_case['H'].values[0] / (2 * k_fl_l)))
            U_gamb = 1 / ((1 / hsky_) + (0.025 / (2 * k_gl)))
            U_absins = 1 / ((0.025 / (2 * k_abs)) + (0.05 / (2 * k_ins)))
            U_insr = 1 / ((1 / hins_) + (0.05 / (2 * k_ins)))
            
            A_[0] = [(rho_gl * cp_gl * V_g_abs / (delta_t * htf_area)) - U_gfl - U_gamb - hrwg_, U_gfl, hrwg_, 0.0]; X_[0] = T_glass_l
            B_[0] = (T_glass_prev * rho_gl * cp_gl * V_g_abs / (delta_t * htf_area)) + irr_glass_ - U_gamb * 300
            A_[1] = [U_gfl, (rho_fl_l * cp_fl_l * V_fluid / (delta_t * htf_area)) - U_gfl - U_absfl, U_absfl, 0.0]; X_[1] = T_fluid_l
            B_[1] = (T_fluid_prev * rho_fl_l * cp_fl_l * V_fluid / (delta_t * htf_area))
            A_[2] = [hrwg_, U_absfl, (rho_abs * cp_abs * V_g_abs / (delta_t * htf_area)) - U_absfl - U_absins - hrwg_, U_absins]; X_[2] = T_abs_l
            B_[2] = (T_abs_prev * rho_abs * cp_abs * V_g_abs / (delta_t * htf_area)) + irr_abs_
            A_[3] = [0, 0, U_absins, (rho_ins * cp_ins * V_ins / (delta_t * htf_area)) - U_absins - U_insr]; X_[3] = T_ins_l
            B_[3] = (T_ins_prev * rho_ins * cp_ins * V_ins / (delta_t * htf_area)) - U_insr * 300
            
            error_ = [ np.abs((B_[x] - y) * 100 / B_[x])  for x, y in enumerate(np.dot(A_, X_))]
            err_g.append(error_[0]); err_fl.append(error_[1]); err_abs.append(error_[2]); err_ins.append(error_[3])

            T_glass_prev = T_glass_l; T_fluid_prev = T_fluid_l; T_abs_prev = T_abs_l; T_ins_prev = T_ins_l

            # empirical type 2
            # f_ = pow(0.79 * math.log(df_case['Re'].values[0]) - 1.64, -2)
            kcoef_ = 1 / df_case['Cd'].values[it1]**2
            try:
                emp_type2 = df_case['Cd'].values[it1] * np.min([rho_fl_l/rho_stream_l, rho_stream_l/rho_fl_l]) * df_case['A'].values[0] * \
                    np.sqrt(9.81 * 2.5 * np.max([T_fluid_l - T_stream_l, 0]) / T_stream_l) * rho_fl_l
                err_type2.append((emp_type2 - df_case['mdot'].values[it1]) * 100 / df_case['mdot'].values[it1])
            except:
                err_type2.append(0)
    
    return np.array(err_type2), np.array(err_g), np.array(err_fl), np.array(err_abs), np.array(err_ins)    

df_['err_type2'], df_['NE_g'], \
df_['NE_stream'], df_['NE_abs'], df_['NE_ins'] = calc_validation(df_)

df_['NE_total'] = df_['NE_stream'] + df_['NE_g'] + df_['NE_abs'] + df_['NE_ins']
q_total = df_['q_stream'] + df_['q_g'] + df_['q_abs'] + df_['q_ins']        
df_['err_total'] = df_['NE_total'] * 100 / q_total

# calc ventilation
# calc performance
# calc relative exposure
    
rt_to_df = []

for case_name, df_case in df_.groupby('case'):
    rt_temp = calc_rt(df_case)
    rt_to_df.extend(rt_temp)

df_['Rt'] = rt_to_df
df_['ACH'] = df_['v_stream'] * df_['A'] * 3600 / V_kamar

# regression vars
psi_ = []; v_dn_ = []
for it1, it2 in df_.groupby('case'):
    init_v0 = it2['v_stream'].values[-1]
    for it3 in list(it2.index):
        theory_gradz = np.sqrt(9.81 * np.max([0, it2.loc[it3,'T_surface'] - it2.loc[it3, 'T_fluid']]) \
                        * it2.loc[it3, 'Dh'] / it2.loc[it3, 'T_fluid'])
        v_dn_.append(theory_gradz)
        get_psi = (it2.loc[it3, 'v_stream'] - init_v0) / (theory_gradz)
        psi_.append(get_psi)
df_['v_dn'] = v_dn_
df_['psi'] = psi_
df_['eta'] = df_['mdot'].values * cp_fl_ * np.array([np.max([0, x]) for x in df_['T_fluid'].values - T0_]) / df_['q'].values
# df_['h'] * df_['W'] * 2.5 * (df_['T_abs'] + df_['T_glass'] - 2 * df_['T_stream']) / (df_['W'] * 2.5 * irr_)

df_['reg_v_y'] = np.array([x/math.exp(-y) for x,y in zip(df_['psi'], df_['NuFo'])])
df_['reg_eta_y'] = np.array([(1 - x)/math.exp(-y) for x,y in zip(df_['eta'], df_['NuFo'])])

os.chdir(f'{caseloc}\\tabel')
df_.to_csv(f'analisis_{casename}.csv', index=False)
print(f'Saved to analisis_{casename}.csv')

Saved to analisis_12-11-23.csv


In [12]:
# model spec
mean_H = 0.575; range_H = 0.425
mean_W = 1.75; range_W = 0.75

channel_H_transform = lambda x: mean_H + (x/1.414) * range_H
channel_W_transform = lambda x: mean_W + (x/1.414) * range_W

channel_H = np.array([0.27, 0.27, 0.88, 0.88, 1, 0.15, 0.575, 0.575, 0.575])
channel_W = np.array([1.22, 2.28, 1.22, 2.28, 1.75, 1.75, 2.5, 1, 1.75])
channel_A = channel_H * channel_W
channel_HW = channel_H / channel_W
channel_H2 = np.array([x**2 for x in channel_H])
channel_W2 = np.array([x**2 for x in channel_W])

# 1,3; 1,6; 6,7; 6,7
model_spec = {'eta_b1': {'reg': ['const', 'H', 'W', 'HW'], 'exclude': [3]}, 
              'eta_b2': {'reg': ['const', 'H', 'W', 'A'], 'exclude': [1,6]},
              'v_b1': {'reg': ['const', 'H', 'W'], 'exclude': [6,7]},
              'v_b2': {'reg': ['const', 'H', 'W'], 'exclude': [6,7]}
              }
dim_dict = {'const': np.ones(shape=(channel_H.shape[0], )), 'H': channel_H - channel_H[-1], 'W': channel_W - channel_W[-1],
            'A': channel_A - channel_A[-1], 'HW': channel_HW - channel_HW[-1],
            'H2': channel_H2 - channel_H2[-1], 'W2': channel_W2 - channel_W2[-1]}

def b_func(var_which, b_coef, H_, W_):
    b_pred = b_coef[0]
    for reg_id, reg_val in enumerate(model_spec[var_which]['reg']):
        b_pred += b_coef[reg_id+1] * dim_dict_func[reg_val](H_, W_)
    return b_pred

dim_dict_func = {'const': lambda H_, W_: 1, 'H': lambda H_, W_: H_ - channel_H[-1], 'W': lambda H_, W_: W_ - channel_W[-1],
                'A': lambda H_, W_: H_ * W_ - channel_A[-1], 'HW': lambda H_, W_: H_ / W_ - channel_HW[-1],
                'H2': lambda H_, W_: H_**2 - channel_H2[-1], 'W2': lambda H_, W_: W_**2 - channel_W2[-1]}


In [8]:
# train models

# curve fitting and ccd regression
# psi = A exp(B * -NuFo)
# eta = A * tanh(B * -NuFo)

vars_trans = {'v': 'psi', 'eta': 'eta'}

def tanh_curve(x_, a_):
    return 2 * a_ * x_

def trans_tanh_curve(x_, a_):
    return (math.exp(a_ * x_) - math.exp(-a_ * x_)) / (math.exp(a_ * x_) + math.exp(-a_ * x_))

def scale_curve(x_, a_):
    return a_ * x_

def curve_compute(df_data, which_, coef_data, anova_data, save_: bool = True):
    vars_trans = {'eta': 'eta', 'v': 'psi'}
    b1_var_all = []; b2_var_all = []; b_pred = []  
    
    get_psi_trans = df_data['psi'].values; get_psi_trans = np.concatenate((get_psi_trans, [0]))
    norm_func_psi = MinMaxScaler(feature_range=(1, math.exp(1))).fit(get_psi_trans.reshape(-1,1))
    
    for case_name, df_case in df_data.groupby('case'):
        df_galat = df_case.shape[0] - 3; df_model = 2
        
        if which_ in ['eta']:
            get_eta_trans = df_case['eta'].values; get_eta_trans = np.concatenate((get_eta_trans, [0]))
            start_norm = np.mean(get_eta_trans) / np.max(get_eta_trans)
            norm_func_eta = MinMaxScaler(feature_range=(0, start_norm)).fit(get_eta_trans.reshape(-1,1))
            
            x_func_rate = df_case['times'].values * -1 / 2.5
            y_func_rate = np.array([math.log(1 - x) for x in norm_func_eta.transform(df_case['eta'].values.reshape(-1,1)).ravel()])
            
            bound_lower_b2 = np.mean(df_case['h'].values) / (1.225 * 1007 * df_case['H'].values[0])
            popt_b2, pcov_b2 = curve_fit(scale_curve, x_func_rate, y_func_rate, bounds=(0, math.inf), p0 = [bound_lower_b2])
            b2_var_all.append(popt_b2[0])
            
            MSE_b2 = np.sum([round(x,5)**2 for x in y_func_rate - x_func_rate * popt_b2[0]]) / (df_galat + 1)
            b2_std = np.sqrt(MSE_b2**2 / np.sum([pow(x - np.mean(x_func_rate), 2) for x in x_func_rate]))
            b2_cor = b2_std * np.std(x_func_rate) / np.std(y_func_rate)
            try:
                b2_t_val = popt_b2[0] * np.sqrt(x_func_rate.shape[0]) / b2_std
                b2_t_pval = stats.t.sf(np.abs(b2_t_val), x_func_rate.shape[0] - 1) * 2
            except:
                b2_t_val = 100
                b2_t_pval = 0
            
            x_func_scale = np.array([1 - math.exp(-popt_b2[0] * x) for x in df_case['times'].values])
            guess_ = np.max([np.mean(df_case['eta'].values / x_func_scale), 0])
            popt_b1, popt_b1 = curve_fit(scale_curve, x_func_scale, df_case['eta'].values, bounds=(0, math.inf), p0=[guess_])
            popt_b1[0][0] = guess_
            b1_var_all.append(popt_b1[0][0])

            MSE_b1 = np.sum([round(x,5)**2 for x in df_case['eta'].values - x_func_scale * popt_b1[0][0]]) / (df_galat + 1) # popt_b1[0][0]]) / (df_galat + 1)
            b1_std = np.sqrt(MSE_b1**2 / np.sum([pow(x - np.mean(x_func_scale), 2) for x in x_func_scale]))
            b1_cor = b1_std * np.std(x_func_scale) / np.std(df_case['eta'].values)
            try:
                b1_t_val = popt_b1[0][0] * np.sqrt(x_func_scale.shape[0]) / b1_std # popt_b1[0][0] * np.sqrt(x_func_scale.shape[0]) / b1_std
                b1_t_pval = stats.t.sf(np.abs(b1_t_val), x_func_scale.shape[0] - 1) * 2
            except:
                b1_t_val = 100
                b1_t_pval = 0
            
            pred_var = np.array([popt_b1[0][0] * (1 - math.exp(-popt_b2[0] * x)) for x in df_case['times'].values])
            ref_mean_var = np.max(pred_var)
            
            res_var = np.array([round(x,5) for x in df_case['eta'].values - pred_var])
            std_ref = np.std(res_var)
            ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
            try:
                sw_val_var, sw_pval_var = stats.shapiro(res_var)
                dw_val_var = smstats.stattools.durbin_watson(res_var)
            except:
                sw_val_var = 100; sw_pval_var = 0
                dw_val_var = 1
            
            SSE_var = np.sum([x**2 for x in res_var])
            SST_var = np.sum([(z - ref_mean_var)**2 for z in pred_var])
            SSM_var = SST_var - SSE_var
            
            try:
                f_val_var = SSM_var * df_galat / (SSE_var * df_model)
                f_pval_var = stats.f.sf(f_val_var, df_model, df_galat)
            except:
                f_val_var = 100
                f_pval_var = 0
            try:
                r2_var = 1 - SSE_var / SST_var
                r2_adj_var = 1 - (1 - r2_var) * (pred_var.shape[0] - 1) / df_galat
            except:
                r2_var = 1
                r2_adj_var = 1     
            
            b_pred.extend(pred_var)
            
            # popt_b1[0][0]
            coef_data.loc[coef_data.shape[0], :] = [case_name, 'eta', 'b1', popt_b1[0][0],
                                                      b1_std, b1_cor, b1_t_val, b1_t_pval]
            coef_data.loc[coef_data.shape[0], :] = [case_name, 'eta', 'b2', popt_b2[0],
                                                      b2_std, b2_cor, b2_t_val, b2_t_pval]
            anova_data.loc[anova_data.shape[0], :] = [case_name, 'eta', SST_var, SSM_var, SSE_var,
                                                        f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                        ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                        dw_val_var, df_model]
            
        elif which_ in ['v']:
            x_func_rate = df_case['times'].values * -1
            y_func_rate = np.array([math.log(x) for x in norm_func_psi.transform(df_case['psi'].values.reshape(-1,1)).ravel()]) - 4 #math.exp(1)

            bound_lower_b2 = np.mean(df_case['h'].values) / (1.225 * 1007 * df_case['H'].values[0])
            popt_b2, pcov_b2 = curve_fit(scale_curve, x_func_rate, y_func_rate, bounds=(0, math.inf), p0 = [bound_lower_b2])
            b2_var_all.append(popt_b2[0])
            
            MSE_b2 = np.sum([round(x,5)**2 for x in y_func_rate - x_func_rate * popt_b2[0]]) / (df_galat + 1)
            b2_std = np.sqrt(MSE_b2**2 / np.sum([pow(x - np.mean(x_func_rate), 2) for x in x_func_rate]))
            b2_cor = b2_std * np.std(x_func_rate) / np.std(y_func_rate)
            try:
                b2_t_val = popt_b2[0] * np.sqrt(x_func_rate.shape[0]) / b2_std
                b2_t_pval = stats.t.sf(np.abs(b2_t_val), x_func_rate.shape[0] - 1) * 2
            except:
                b2_t_val = 100
                b2_t_pval = 0
            
            x_func_scale = np.array([math.exp(-popt_b2[0] * x) for x in df_case['times'].values])
            guess_ = np.max([np.mean(df_case['psi'].values / x_func_scale), 0])
            popt_b1, pcov_b1 = curve_fit(scale_curve, x_func_scale, df_case['psi'].values, bounds=(0, math.inf), p0 = [guess_])
            # popt_b1[0] = guess_
            b1_var_all.append(popt_b1[0])
            
            MSE_b1 = np.sum([round(x,5)**2 for x in df_case['psi'].values - x_func_scale * popt_b1[0]]) / (df_galat + 1)
            b1_std = np.sqrt(MSE_b1**2 / np.sum([pow(round(x, 5) - np.mean(x_func_scale), 2) for x in x_func_scale]))
            b1_cor = b1_std * np.std(x_func_scale) / np.std(df_case['psi'].values)
            try:
                b1_t_val = popt_b1[0] * np.sqrt(x_func_scale.shape[0]) / b1_std
                b1_t_pval = stats.t.sf(np.abs(b1_t_val), x_func_scale.shape[0] - 1) * 2
            except:
                b1_t_val = 100
                b1_t_pval = 0
            
            pred_var = np.array([popt_b1[0] * math.exp(-popt_b2[0] * x) for x in df_case['times'].values])
            ref_mean_var = np.max(pred_var)
            
            res_var = np.array([round(x,5) for x in df_case['psi'].values - pred_var])
            std_ref = np.std(res_var)
            ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
            try:
                sw_val_var, sw_pval_var = stats.shapiro(res_var)
                dw_val_var = smstats.stattools.durbin_watson(res_var)
            except:
                sw_val_var = 100; sw_pval_var = 0
                dw_val_var = 1
            
            SSE_var = np.sum([x**2 for x in res_var])
            SST_var = np.sum([(z - ref_mean_var)**2 for z in pred_var])
            SSM_var = SST_var - SSE_var
            
            try:
                f_val_var = SSM_var * df_galat / (SSE_var * df_model)
                f_pval_var = stats.f.sf(f_val_var, df_model, df_galat)
            except:
                f_val_var = 100
                f_pval_var = 0
            try:
                r2_var = 1 - SSE_var / SST_var
                r2_adj_var = 1 - (1 - r2_var) * (pred_var.shape[0] - 1) / df_galat
            except:
                r2_var = 1
                r2_adj_var = 1
                
            b_pred.extend(pred_var)
                        
            coef_data.loc[coef_data.shape[0], :] = [case_name, 'v', 'b1', popt_b1[0],
                                                      b1_std, b1_cor, b1_t_val, b1_t_pval]
            coef_data.loc[coef_data.shape[0], :] = [case_name, 'v', 'b2', popt_b2[0],
                                                      b2_std, b2_cor, b2_t_val, b2_t_pval]
            anova_data.loc[anova_data.shape[0], :] = [case_name, 'v', SST_var, SSM_var, SSE_var,
                                                        f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                        ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                        dw_val_var, df_model]
    
    df_data[f'{vars_trans[which_]}_kurva'] = b_pred 
    
    return np.array(b1_var_all).ravel(), np.array(b2_var_all).ravel()

def model_compute(df_data, save_: bool=True):
    # eta = 1 - c exp^{-NuFo}
    # v_stream / v_dn = (GrCf)^n

    coef_curve = pd.DataFrame(columns=['case', 'var', 'coef', 'effect', 'std', 'cor', 't_val', 't_pval'])
    anova_curve = pd.DataFrame(columns=['case', 'var', 'SST', 'SSM', 'SSE', 'f_val', 'f_pval',
                                        'r2', 'r2_adj', 'ks_val', 'ks_pval', 'sw_val', 'sw_pval',
                                        'dw_val', 'df_model'])
    
    # commons
    coef_model = pd.DataFrame(columns=['var', 'seq', 'coef', 'effect', 'std', 'cor', 't_val', 't_pval'])
    anova_scale_model = pd.DataFrame(columns=['var', 'seq', 'SST', 'SSM', 'SSE', 'f_val', 'f_pval',
                                            'r2', 'r2_adj', 'ks_val', 'ks_pval', 'sw_val', 'sw_pval',
                                            'dw_val', 'df_model'])
    anova_rate_model = pd.DataFrame(columns=['var', 'seq', 'SST', 'SSM', 'SSE', 'f_val', 'f_pval',
                                            'r2', 'r2_adj', 'ks_val', 'ks_pval', 'sw_val', 'sw_pval',
                                            'dw_val', 'df_model'])
    # inference-specific anovas
    # psi, eta, v_stream, ACH, Rt, T, h, Gr
    anova_all_model = pd.DataFrame(columns=['var', 'seq', 'SST', 'SSM', 'SSE', 'f_val', 'f_pval',
                                            'r2', 'r2_adj', 'ks_val', 'ks_pval', 'sw_val', 'sw_pval',
                                            'dw_val', 'df_model'])
    
    collect_curve_pred = []
    
    for vars_ in ['eta', 'v']:
        exog_name_b1 = model_spec[f'{vars_}_b1']['reg']; exog_data_b1 = np.ones(shape=(channel_H.shape[0] - len(model_spec[f'{vars_}_b1']['exclude']),))
        exog_name_b2 = model_spec[f'{vars_}_b2']['reg']; exog_data_b2 = np.ones(shape=(channel_H.shape[0] - len(model_spec[f'{vars_}_b2']['exclude']),))
        
        for it1 in model_spec[f'{vars_}_b1']['reg']:
            to_append = np.array([y for x,y in enumerate(dim_dict[it1]) if x not in model_spec[f'{vars_}_b1']['exclude']])
            exog_data_b1 = np.vstack((exog_data_b1, to_append))
        exog_data_b1 = exog_data_b1[1:]; exog_data_b1 = exog_data_b1.T
        
        for it1 in model_spec[f'{vars_}_b2']['reg']:
            to_append = np.array([y for x,y in enumerate(dim_dict[it1]) if x not in model_spec[f'{vars_}_b2']['exclude']])
            exog_data_b2 = np.vstack((exog_data_b2, to_append))
        exog_data_b2 = exog_data_b2[1:]; exog_data_b2 = exog_data_b2.T
        
        b1_var_all, b2_var_all = curve_compute(df_data, vars_, coef_curve, anova_curve)
        
        # decentralize
        b1_var = np.array([x - b1_var_all[-1] for x in b1_var_all]); b2_var = np.array([x - b2_var_all[-1] for x in b2_var_all])
        
        b1_var = np.array([y for x,y in enumerate(b1_var) if x not in model_spec[f'{vars_}_b1']['exclude']])
        b2_var = np.array([y for x,y in enumerate(b2_var) if x not in model_spec[f'{vars_}_b2']['exclude']])
        
        b1_var_norm_func = StandardScaler(with_mean=False, with_std=False).fit(b1_var.reshape(-1,1))
        b2_var_norm_func = StandardScaler(with_mean=False, with_std=False).fit(b2_var.reshape(-1,1))
        
        aktual_var = np.array([df_data[df_data['case'] == x].loc[:,vars_trans[vars_]].values for x in range(1,10)])
        ref_var = aktual_var[-1]
        
        df_model_b1 = len(model_spec[f'{vars_}_b1']['reg']); df_model_b2 = len(model_spec[f'{vars_}_b2']['reg'])
        df_galat_b1 = df_data.shape[0] - df_model_b1 - 1; df_galat_b2 = df_data.shape[0] - df_model_b2 - 1
        df_model = df_model_b1 + df_model_b2; df_galat = df_data.shape[0] - df_model - 1
        
        # print(b1_var_all, b2_var_all)
        
        # rs modelling
        
        b1_train = b1_var_norm_func.transform(b1_var.reshape(-1,1)).ravel()
        b2_train = b2_var_norm_func.transform(b2_var.reshape(-1,1)).ravel()
        
        # b1
        uwls_var_b1 = sm.OLS(b1_train.astype(float), exog_data_b1.astype(float), missing=('drop')).fit()
        res_var_b1 = np.array(uwls_var_b1.resid)
        weights_b1 = [1 / x**2 for x in res_var_b1]
        wls_var_b1 = sm.WLS(b1_train.astype(float), exog_data_b1.astype(float), weights_b1, missing=('drop')).fit()
        # wls_var_b1 = sm.WLS(b1_var.astype(float), exog_data_b1.astype(float), missing=('drop')).fit()
        
        # b2
        uwls_var_b2 = sm.OLS(b2_train.astype(float), exog_data_b2.astype(float), missing=('drop')).fit()
        res_var_b2 = np.array(uwls_var_b2.resid)
        weights_b2 = [1 / x**2 for x in res_var_b2]
        wls_var_b2 = sm.WLS(b2_train.astype(float), exog_data_b2.astype(float), weights_b2, missing=('drop')).fit()
        # wls_var_b2 = sm.WLS(b2_var.astype(float), exog_data_b2.astype(float), missing=('drop')).fit()
        
        coef_name_in_l_b1 = wls_var_b1.model.exog_names
        
        check_var = []; [check_var.extend(x - ref_var) for x in aktual_var]; check_var = np.array(check_var)
        
        coef_model.loc[coef_model.shape[0],:] = [vars_, 'b1', 'interp', b1_var_all[-1], 0, 0, 0, 0]
        coef_model.loc[coef_model.shape[0],:] = [vars_, 'b2', 'interp', b2_var_all[-1], 0, 0, 0, 0]
        
        for it1, it2 in enumerate(coef_name_in_l_b1):
            efek_b1 = b1_var_norm_func.inverse_transform([[wls_var_b1.t_test(it2).effect[0]]]).ravel()[0]
            efek_b2 = b2_var_norm_func.inverse_transform([[wls_var_b2.t_test(it2).effect[0]]]).ravel()[0]
            sd_b1 = b1_var_norm_func.inverse_transform([[wls_var_b1.t_test(it2).sd[0][0]]]).ravel()[0]
            sd_b2 = b2_var_norm_func.inverse_transform([[wls_var_b2.t_test(it2).sd[0][0]]]).ravel()[0]
            try:
                b1_t_val = efek_b1 * np.sqrt(df_galat_b1) / sd_b1
                b1_t_pval = stats.t.sf(np.abs(b1_t_val), df_galat_b1) * 2
                b2_t_val = efek_b2 * np.sqrt(df_galat_b2) / sd_b2
                b2_t_pval = stats.t.sf(np.abs(b2_t_val), df_galat_b2) * 2
            except:
                b2_t_val = 100
                b2_t_pval = 0
            
            try:
                cor_b1 = efek_b1 * np.std(exog_data_b1[it1]) / np.std(check_var)
            except:
                cor_b1 = 1
            try:
                cor_b2 = efek_b2 * np.std(exog_data_b2[it1]) / np.std(check_var)
            except:
                cor_b2 = 1
                
            # coef_model.loc[coef_model.shape[0],:] = [vars_, 'b1', it2, efek_b1, sd_b1,
            #                                         cor_b1, wls_var_b1.t_test(it2).tvalue[0][0],
            #                                         float(wls_var_b1.t_test(it2).pvalue)]
            # coef_model.loc[coef_model.shape[0],:] = [vars_, 'b2', it2, efek_b2, sd_b2,
            #                                         cor_b2, wls_var_b2.t_test(it2).tvalue[0][0],
            #                                         float(wls_var_b2.t_test(it2).pvalue)]
            coef_model.loc[coef_model.shape[0],:] = [vars_, 'b1', it2, efek_b1, sd_b1,
                                                    cor_b1, b1_t_val, b1_t_pval]
            coef_model.loc[coef_model.shape[0],:] = [vars_, 'b2', it2, efek_b2, sd_b2,
                                                    cor_b2, b2_t_val, b2_t_pval]
        
        # calc model eval
        # sequence
        
        exog_name_b1 = model_spec[f'{vars_}_b1']['reg']; exog_data_b1 = np.ones(shape=(channel_H.shape[0],))
        exog_name_b2 = model_spec[f'{vars_}_b2']['reg']; exog_data_b2 = np.ones(shape=(channel_H.shape[0],))
        
        for it1 in model_spec[f'{vars_}_b1']['reg']:
            exog_data_b1 = np.vstack((exog_data_b1, dim_dict[it1]))
        exog_data_b1 = exog_data_b1[1:]; exog_data_b1 = exog_data_b1.T
        for it1 in model_spec[f'{vars_}_b2']['reg']:
            exog_data_b2 = np.vstack((exog_data_b2, dim_dict[it1]))
        exog_data_b2 = exog_data_b2[1:]; exog_data_b2 = exog_data_b2.T
        
        for var_id, var_name in enumerate(exog_name_b1):
            # predict sequence
            exog_anova_b1 = deepcopy(exog_data_b1.T)
            exog_anova_b2 = deepcopy(exog_data_b2.T)
            isolate = list(range(var_id+1, len(exog_name_b1)))  
            if len(isolate) > 0:
                for varmodifyid in isolate:
                    exog_anova_b1[varmodifyid] = np.zeros((1,exog_anova_b1.shape[1])).ravel()
                    exog_anova_b2[varmodifyid] = np.zeros((1,exog_anova_b2.shape[1])).ravel()
            exog_anova_b1 = exog_anova_b1.T
            exog_anova_b2 = exog_anova_b2.T
            
            pred_var_b1 = np.array([wls_var_b1.predict(exog=x) + b1_var_all[-1] for x in exog_anova_b1]).ravel()
            pred_var_b2 = np.array([wls_var_b2.predict(exog=x) + b2_var_all[-1] for x in exog_anova_b2]).ravel()
            pred_var_b_all = []; pred_var_specific_all = {}
            pred_var_q = []
            
            if vars_ in ['eta']:
                pred_var_T = [] # T_fluid
                pred_var_h = []
                for case_name, df_case in df_data.groupby('case'):
                    case_ids = list(df_case.index)
                    # heat_area = df_case['W'].values[0] * 2.5
                    eta_temp = [ pred_var_b1[case_name-1] * (1 - math.exp(-pred_var_b2[case_name-1] * x)) \
                                            for x in df_case['times'].values ]
                    pred_var_b_all.append(eta_temp); eta_temp = np.array(eta_temp)
                    pred_var_T.append([ a * d / (b * c) + T0_ for a, b, c, d in zip(eta_temp, df_case['mdot'].values, cp_fl_[case_ids], df_case['q'].values) ])
                    pred_var_h.append([ pred_var_b2[case_name-1] * 1.225 * 1007 * df_case['H'].values[0] for x in case_ids])
                    pred_var_q.append([ pred_var_b1[case_name-1] * x for x in df_case['q'].values ])
                pred_var_specific_all['T_fluid'] = np.array(pred_var_T).ravel()
                pred_var_specific_all['h_eta'] = np.array(pred_var_h).ravel()
                pred_var_specific_all['q'] = np.array(pred_var_q).ravel()
                
            elif vars_ in ['v']:
                pred_var_v = []
                pred_var_ach = []
                pred_var_rt = []
                pred_var_h = []
                pred_var_vdn = []
                # pred_var_gr = [] # after T_fluid approximated
                for case_name, df_case in df_data.groupby('case'):
                    case_ids = list(df_case.index)
                    case_area = df_case['A'].values[0]
                    init_v0 = df_case['v_stream'].values[-1]
                    psi_temp = [ pred_var_b1[case_name-1] * math.exp(-pred_var_b2[case_name-1] * x) for \
                                        x in df_case['times'].values ]
                    delta_T_temp = df_case['q_model'].values / (df_case['h_eta_model'].values * df_case['W'].values[0] * 2.5)
                    # v_dn_temp = np.array([ np.sqrt(9.81 * df_case['Dh'].values[0] * x / y) for x,y in zip(delta_T_temp, df_case['T_fluid_model'].values) ])
                    v_dn_temp = []
                    for delta_T_, T_fl_ in zip(delta_T_temp, df_case['T_fluid_model'].values):
                        try:
                            v_dn_temp.append(pow(9.81 * df_case['Dh'].values[0] * delta_T_ / T_fl_, 0.5))
                        except:
                            v_dn_temp.append(0)
                    v_temp = [ x * y + init_v0 for x,y in zip(psi_temp, v_dn_temp) ]
                    pred_var_b_all.append([ psi_temp ])
                    pred_var_v.append(v_temp)
                    pred_var_ach.append([ x * case_area * 3600 / (V_kamar) for x in v_temp ])
                    rt_temp = calc_rt(df_case)
                    pred_var_rt.append(rt_temp)
                    pred_var_h.append([ pred_var_b2[case_name-1] * 1.225 * 1007 * df_case['H'].values[0] for x in case_ids])
                    pred_var_vdn.append(v_dn_temp)
                pred_var_specific_all['v_stream'] = np.array(pred_var_v).ravel()
                pred_var_specific_all['ACH'] = np.array(pred_var_ach).ravel()
                pred_var_specific_all['Rt'] = np.array(pred_var_rt).ravel()
                pred_var_specific_all['h_psi'] = np.array(pred_var_h).ravel()
                pred_var_specific_all['v_dn'] = np.array(pred_var_vdn).ravel()
            
            pred_var_b_all = np.array(pred_var_b_all).ravel()

            if len(isolate) == 0:
                [collect_curve_pred.extend([x,y]) for x,y in zip(pred_var_b1, pred_var_b2)]
                df_data[f'{vars_trans[vars_]}_model'] = pred_var_b_all
                for it1, it2 in pred_var_specific_all.items():
                    df_data[f'{it1}_model'] = it2
                # if vars_ in ['eta']:
                #     df_data['q_model'] = pred_var_q
                # elif vars_ in ['v']:
                #     df_data['v_dn_model'] = pred_var_vdn
            
            # calc eval
            ref_mean_b1 = np.median(b1_var_all)
            ref_mean_b2 = np.median(b2_var_all)
            ref_mean_resp = np.median(df_data[vars_trans[vars_]].values) # np.median([np.median(y[vars_trans[vars_]].values) for x, y in df_data.groupby('case')])
            
            # scale
            res_var = np.array([x for x in b1_var_all - pred_var_b1])
            std_ref = np.std(res_var)
            ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
            try:
                sw_val_var, sw_pval_var = stats.shapiro(res_var)
                dw_val_var = smstats.stattools.durbin_watson(res_var)
            except:
                sw_val_var = 100; sw_pval_var = 0
                dw_val_var = 1
            
            SSE_var = np.sum([x**2 for x in res_var])
            SST_var = np.sum([(z - ref_mean_b1)**2 for z in pred_var_b1])
            SSM_var = SST_var - SSE_var
            
            try:
                f_val_var = SSM_var * df_galat_b1 / (SSE_var * df_model_b1)
                f_pval_var = stats.f.sf(f_val_var, df_model_b1, df_galat_b1)
            except:
                f_val_var = 100
                f_pval_var = 0
            try:
                r2_var = 1 - SSE_var / SST_var
                r2_adj_var = 1 - (1 - r2_var) * (df_data.shape[0] - 1) / df_galat_b1
            except:
                r2_var = 1
                r2_adj_var = 1
            
            anova_scale_model.loc[anova_scale_model.shape[0], :] = [vars_, var_name, SST_var, SSM_var, SSE_var,
                                                                f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                                ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                                dw_val_var, df_model_b1]
            
            # rate
            res_var = np.array([x for x in b2_var_all - pred_var_b2])
            std_ref = np.std(res_var)
            ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
            try:
                sw_val_var, sw_pval_var = stats.shapiro(res_var)
                dw_val_var = smstats.stattools.durbin_watson(res_var)
            except:
                sw_val_var = 100; sw_pval_var = 0
                dw_val_var = 1
                            
            SSE_var = np.sum([x**2 for x in res_var])
            SST_var = np.sum([(z - ref_mean_b2)**2 for z in b2_var_all])
            SSM_var = SST_var - SSE_var
            
            try:
                f_val_var = SSM_var * df_galat_b2 / (SSE_var * df_model_b2)
                f_pval_var = stats.f.sf(f_val_var, df_model_b2, df_galat_b2)
            except:
                f_val_var = 100
                f_pval_var = 0
            try:
                r2_var = 1 - SSE_var / SST_var
                r2_adj_var = 1 - (1 - r2_var) * (df_data.shape[0] - 1) / df_galat_b2
            except:
                r2_var = 1
                r2_adj_var = 1
            
            anova_rate_model.loc[anova_rate_model.shape[0], :] = [vars_, var_name, SST_var, SSM_var, SSE_var,
                                                                f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                                ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                                dw_val_var, df_model_b2]
            
            # all
            res_var = np.array([x for x in df_data[vars_trans[vars_]].values - pred_var_b_all])
            std_ref = np.std(res_var)
            ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
            try:
                sw_val_var, sw_pval_var = stats.shapiro(res_var)
                dw_val_var = smstats.stattools.durbin_watson(res_var)
            except:
                sw_val_var = 100; sw_pval_var = 0
                dw_val_var = 1
            
            SSE_var = np.sum([x**2 for x in res_var])
            SST_var = np.sum([(z - ref_mean_resp)**2 for z in df_data[vars_trans[vars_]].values])
            SSM_var = SST_var - SSE_var
            
            try:
                f_val_var = SSM_var * df_galat / (SSE_var * df_model)
                f_pval_var = stats.f.sf(f_val_var, df_model, df_galat)
            except:
                f_val_var = 100
                f_pval_var = 0
            try:
                r2_var = 1 - SSE_var / SST_var
                r2_adj_var = 1 - (1 - r2_var) * (df_data.shape[0] - 1) / df_galat
            except:
                r2_var = 1
                r2_adj_var = 1
            
            anova_all_model.loc[anova_all_model.shape[0], :] = [vars_, var_name, SST_var, SSM_var, SSE_var,
                                                                f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                                ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                                dw_val_var, df_model]
            
            # specifics
            for specific_name, specific_resp in pred_var_specific_all.items():
                ref_mean_resp = np.max(specific_resp)
                
                res_var = df_data[specific_name].values - specific_resp
                std_ref = np.std(res_var)
                ks_val_var, ks_pval_var = stats.ks_2samp(res_var, stats.norm.rvs(size=res_var.shape[0], loc=0, scale=std_ref))
                try:
                    sw_val_var, sw_pval_var = stats.shapiro(res_var)
                    dw_val_var = smstats.stattools.durbin_watson(res_var)
                except:
                    sw_val_var = 100; sw_pval_var = 0
                    dw_val_var = 1
                
                SSE_var = np.sum([x**2 for x in res_var])
                SST_var = np.sum([(z - ref_mean_resp)**2 for z in specific_resp])
                SSM_var = SST_var - SSE_var
                
                try:
                    f_val_var = SSM_var * df_galat / (SSE_var * df_model)
                    f_pval_var = stats.f.sf(f_val_var, df_model, df_galat)
                except:
                    f_val_var = 100
                    f_pval_var = 0
                try:
                    r2_var = 1 - SSE_var / SST_var
                    r2_adj_var = 1 - (1 - r2_var) * (specific_resp.shape[0] - 1) / df_galat
                except:
                    r2_var = 1
                    r2_adj_var = 1
                
                anova_all_model.loc[anova_all_model.shape[0], :] = [specific_name, var_name, SST_var, SSM_var, SSE_var,
                                                                    f_val_var, f_pval_var, r2_var, r2_adj_var,
                                                                    ks_val_var, ks_pval_var, sw_val_var, sw_pval_var,
                                                                    dw_val_var, df_model]

    coef_curve['pred'] = collect_curve_pred
            
    if save_ is True:
        if not os.path.exists(f'{caseloc}\\tabel'):
            try:
                original_umask = os.umask(0)
                os.makedirs(f'{caseloc}\\tabel', 0o777)
            finally:
                os.umask(original_umask)
        os.chdir(f'{caseloc}\\tabel')
        # print('Saving results...')
        coef_curve.to_csv(f'coef_curve_{casename}.csv')
        anova_curve.to_csv(f'anova_curve_{casename}.csv')
        coef_model.to_csv(f'coef_rs_{casename}.csv')
        anova_scale_model.to_csv(f'anova_rs_scale_{casename}.csv')
        anova_rate_model.to_csv(f'anova_rs_rate_{casename}.csv')
        anova_all_model.to_csv(f'anova_rs_all_{casename}.csv')
        print(f'Saved to {os.getcwd()}')  
    
    return df_data

df_ = model_compute(df_)
df_['Nu_eta_model'] = df_['h_eta_model'].values * df_['Dh'].values / 0.025
df_['Nu_psi_model'] = df_['h_psi_model'].values * df_['Dh'].values / 0.025

os.chdir(f'{caseloc}\\tabel')
df_.to_csv(f'analisis_{casename}.csv', index=False)
print(f'Saved to analisis_{casename}.csv')

c:\Users\LENOVO\Documents\TA\Repo\.venv\Lib\site-packages\scipy\stats\_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\Users\LENOVO\Documents\TA\Repo\.venv\Lib\site-packages\scipy\stats\_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\Users\LENOVO\Documents\TA\Repo\.venv\Lib\site-packages\scipy\stats\_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\Users\LENOVO\Documents\TA\Repo\.venv\Lib\site-packages\scipy\stats\_morestats.py:1813: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "
c:\Users\LENOVO\Docu

Saved to C:\Users\LENOVO\Documents\TA\Data\12-11-23\tabel
Saved to analisis_12-11-23.csv


In [18]:
# plot, everything
# import no calc
casename = '12-11-23'
caseloc = f'C:\\Users\\LENOVO\\Documents\\TA\\Data\\{casename}'
irr_ = 500; v0_ = 1e-2; T0_ = 300
V_tot = 0.15 * 3**2 + 3.5
V_kamar = 3**2 * 2.5
y_offset_fontsize = 15

os.chdir(f'{caseloc}\\tabel')
df_ = pd.read_csv(f'analisis_{casename}.csv')
df_coef = pd.read_csv(f'coef_rs_{casename}.csv')
df_curve = pd.read_csv(f'coef_curve_{casename}.csv')

def trans_tanh_curve(x_, a_):
    return (math.exp(a_ * x_) - math.exp(-a_ * x_)) / (math.exp(a_ * x_) + math.exp(-a_ * x_))

coef_rs_l = {'eta_b1': df_coef[(df_coef['var'] == 'eta') & (df_coef['seq'] == 'b1')].loc[:, 'effect'].values,
             'eta_b2': df_coef[(df_coef['var'] == 'eta') & (df_coef['seq'] == 'b2')].loc[:, 'effect'].values,
             'v_b1': df_coef[(df_coef['var'] == 'v') & (df_coef['seq'] == 'b1')].loc[:, 'effect'].values,
             'v_b2': df_coef[(df_coef['var'] == 'v') & (df_coef['seq'] == 'b2')].loc[:, 'effect'].values}

coef_curve_l = {'eta_b1': df_curve[(df_curve['var'] == 'eta') & (df_curve['coef'] == 'b1')].loc[:, 'effect'].values,
                'eta_b2': df_curve[(df_curve['var'] == 'eta') & (df_curve['coef'] == 'b2')].loc[:, 'effect'].values,
                'v_b1': df_curve[(df_curve['var'] == 'v') & (df_curve['coef'] == 'b1')].loc[:, 'effect'].values,
                'v_b2': df_curve[(df_curve['var'] == 'v') & (df_curve['coef'] == 'b2')].loc[:, 'effect'].values}

mean_H = 0.575; range_H = 0.425
mean_W = 1.75; range_W = 0.75

channel_H_transform = lambda x: mean_H + x * range_H
channel_W_transform = lambda x: mean_W + x * range_W

channel_H = np.array([0.27, 0.27, 0.88, 0.88, 1, 0.15, 0.575, 0.575, 0.575])
channel_W = np.array([1.22, 2.28, 1.22, 2.28, 1.75, 1.75, 2.5, 1, 1.75])
channel_A = channel_H * channel_W
channel_H2 = np.array([x**2 for x in channel_H])
channel_W2 = np.array([x**2 for x in channel_W])

# functions
scos = lambda x: sp.N(sp.cos(x))
ssin = lambda x: sp.N(sp.sin(x))

fmt = lambda x: '{:n}'.format(round(x,2))

coef_rs_pred_l = {'eta_b1': [b_func('eta_b1', coef_rs_l['eta_b1'], df_case['H'].values[0], df_case['W'].values[0]) for case_name, df_case in df_.groupby('case')],
                  'eta_b2': [b_func('eta_b2', coef_rs_l['eta_b2'], df_case['H'].values[0], df_case['W'].values[0]) for case_name, df_case in df_.groupby('case')],
                  'v_b1': [b_func('v_b1', coef_rs_l['v_b1'], df_case['H'].values[0], df_case['W'].values[0]) for case_name, df_case in df_.groupby('case')],
                  'v_b2': [b_func('v_b2', coef_rs_l['v_b2'], df_case['H'].values[0], df_case['W'].values[0]) for case_name, df_case in df_.groupby('case')]}

# predict inference
coef_func_eta = lambda b1_coef, b2_coef, H_, W_, t_: b_func('eta_b1', b1_coef, H_, W_) * (1 - math.exp(-b_func('eta_b2', b2_coef, H_, W_) * t_))
coef_func_v = lambda b1_coef, b2_coef, H_, W_, t_: b_func('v_b1', b1_coef, H_, W_) * math.exp(-b_func('v_b2', b2_coef, H_, W_) * t_)

# specific derivatives
coef_func_T_fluid = lambda coef_rs_l, H_, W_, t_: coef_func_eta(coef_rs_l['eta_b1'], coef_rs_l['eta_b2'], H_, W_, t_) * irr_ * 2.5 * W_ / (1.225 * 0.01 * H_ * W_ * 1007)
coef_func_h = lambda coef_rs_l, H_, W_, t_: b_func('eta_b2', coef_rs_l['eta_b2'], H_, W_) * 1.225 * 1007 * H_
coef_func_h_psi = lambda coef_rs_l, H_, W_, t_: b_func('v_b2', coef_rs_l['v_b2'], H_, W_) * 1.225 * 1007 * H_
coef_func_nu = lambda coef_rs_l, H_, W_, t_: coef_func_h(coef_rs_l, H_, W_, t_) * (2 * H_ * W_ / (H_ + W_)) / 0.025
coef_func_nu_psi = lambda coef_rs_l, H_, W_, t_: coef_func_h_psi(coef_rs_l, H_, W_, t_) * (2 * H_ * W_ / (H_ + W_)) / 0.025
coef_func_q = lambda coef_rs_l, H_, W_, t_: b_func('eta_b1', coef_rs_l['eta_b1'], H_, W_) * irr_ * 2.5 * W_

def coef_func_v_stream(coef_rs_l, H_, W_, t_):
    psi_temp = coef_func_v(coef_rs_l['v_b1'], coef_rs_l['v_b2'], H_, W_, t_)
    Dh_temp = 2 * H_ * W_ / (H_ + W_)
    T_temp = coef_func_T_fluid(coef_rs_l, H_, W_, t_) + T0_
    h_temp = coef_func_h(coef_rs_l, H_, W_, t_)
    b1_temp = b_func('eta_b1', coef_rs_l['eta_b1'], H_, W_)
    T_delta_temp = np.max([b1_temp * irr_ / h_temp, 0])
    vdn_temp = np.sqrt(9.81 * Dh_temp * T_delta_temp / T_temp)
    v_stream_temp = psi_temp * vdn_temp
    return v_stream_temp
coef_func_ach = lambda coef_rs_l, H_, W_, t_: (coef_func_v_stream(coef_rs_l, H_, W_, t_) + v0_) * H_ * W_ * 3600 / V_kamar
def coef_func_vdn(coef_rs_l, H_, W_, t_):
    T_temp = coef_func_T_fluid(coef_rs_l, H_, W_, t_) + T0_
    Dh_temp = 2 * H_ * W_ / (H_ + W_)
    h_temp = coef_func_h(coef_rs_l, H_, W_, t_)
    b1_temp = b_func('eta_b1', coef_rs_l['eta_b1'], H_, W_)
    T_delta_temp = np.max([b1_temp * irr_ / h_temp, 0])
    # b_psi_temp = b_func('v_b1', coef_rs_l['v_b1'], H_, W_) 
    v_dn_temp = np.sqrt(9.81 * Dh_temp * T_delta_temp / T_temp)
    return v_dn_temp

def coef_func_rt(coef_rs_l, H_, W_, t_):
    Vdot_ = np.array([coef_func_v_stream(coef_rs_l, H_, W_, x) + v0_ for x in np.linspace(0, 3600, 62, endpoint=True)]) * H_ * W_
    rt_ = [(V_tot / Vdot_[0]) + (1 - (V_tot / Vdot_[0])) * math.exp(-Vdot_[0] * delta_t / V_kamar)]
    for it1, it2 in enumerate(Vdot_[1:]):
        rt_.append((V_tot / it2) + (rt_[-1] - (V_tot / it2)) * math.exp( -it2 * delta_t / V_kamar))
    if int(t_) < 3200:
        return rt_[int(math.floor(len(rt_)/2))]
    else:
        return rt_[-1]

coef_func = {'eta': coef_func_eta, 'v': coef_func_v,
             'T_fluid': coef_func_T_fluid, 'h_eta': coef_func_h, 'h_psi': coef_func_h_psi, 'Nu_eta': coef_func_nu, 'Nu_psi': coef_func_nu_psi, 'q': coef_func_q,
             'v_stream': coef_func_v_stream, 'ACH': coef_func_ach, 'Rt': coef_func_rt, 'v_dn': coef_func_vdn
             }

vars_trans = {'eta': 'eta', 'v': 'psi'}
time_this_trans = {'times': 'time', 'NuFo': 'nufo'}

def plot_vs_time(df_smooth_data, save_:bool = True):
    
    vars_l = ['v_stream', 'Cf', 'Pgrady_stream', 'T_fluid', 'T_abs', 'T_glass', 'T_insul', 'h', 'Nu', 'Gr', 'Re', 'Ri',
              'Rt', 'ACH', 'psi', 'eta', 'reg_v_y', 'reg_eta_y', 'NuFo']

    # vars_l = ['T_glass', 'T_insul', 'h']
    
    df_compound = deepcopy(df_smooth_data)
    
    df_compound['v_stream'] = df_compound['v_stream'] - v0_
    df_compound['T_fluid'] = df_compound['T_fluid'] - T0_
    df_compound['T_abs'] = df_compound['T_abs'] - T0_
    df_compound['T_glass'] = df_compound['T_glass'] - T0_
    df_compound['T_insul'] = df_compound['T_insul'] - T0_
    df_compound['Nu'] = df_compound['Nu'] - 1
    
    df_compound['Ri'] = [x/pow(y,2) for x,y in zip(df_compound['Gr'].values, df_compound['Re'].values)]

    vars_minmax = dict(zip(vars_l, [[np.min(df_compound[x].values), np.max(df_compound[x].values)] for x in vars_l]))
    vars_offset = dict(zip(vars_l, [int(np.max([math.floor(math.log10(np.abs(x[0]))), math.floor(math.log10(np.abs(x[1])))])) for x in list(vars_minmax.values())]))
    vars_minmax = dict(zip(vars_l, [[(vars_minmax[x][0] - 0.15 * (vars_minmax[x][1] - vars_minmax[x][0])) * pow(10, -y),
                                     (vars_minmax[x][1] + 0.15 * (vars_minmax[x][1] - vars_minmax[x][0])) * pow(10, -y)] for x,y in vars_offset.items()]))
    
    # plot vs time plot
    for case_id, df_case in df_compound.groupby('case'):
        for vars_ in vars_l:
            fig_vs_time = plt.figure(figsize=(3,3))
            ax_vs_time = fig_vs_time.add_subplot(1,1,1)

            ax_vs_time.plot(df_case['times'].values, df_case[vars_].values * pow(10, -vars_offset[vars_]), color='black', linestyle='solid', label=case_id)
           
            ax_vs_time.text(0.02, 1.1, r'$\times 10^{' + f'{vars_offset[vars_]}' + r'}$',
                    horizontalalignment='left', verticalalignment='top',
                    transform=ax_vs_time.transAxes)
                        
            labelLines(ax_vs_time.get_lines(), align=False)
            
            ax_vs_time.set_ylim(vars_minmax[vars_])
            
            ax_vs_time.set_xlabel('Waktu [s]', labelpad=10, horizontalalignment='center')
            ax_vs_time.set_ylabel('')
            ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])
            
            ax_vs_time2 = ax_vs_time.twinx()
            ax_vs_time2.set_yticks([round(df_case[vars_].values[-1] * pow(10, -vars_offset[vars_]),2)])
            ax_vs_time2.set_ylim(ax_vs_time.get_ylim())
            
            if save_ is True:
                if not os.path.exists(f'{caseloc}\\plot\\obv'):
                    try:
                        original_umask = os.umask(0)
                        os.makedirs(f'{caseloc}\\plot\\obv', 0o777)
                    finally:
                        os.umask(original_umask)
                os.chdir(f'{caseloc}\\plot\\obv')
                # print('Plotting results...')
                fig_vs_time.savefig(f'{vars_}_{case_id}_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
                print('Saved to x_plot.pdf')
                plt.close(fig_vs_time)

# plot error and validation
def plot_error(df_data, save_: bool = True):
    dict_rename = {'NE_stream': 'ud.', 'NE_g': 'kaca', 'NE_abs': 'abs.', 'NE_ins': 'ins.'}
    
    # NE
    fig_NE = plt.figure(figsize=(3.75,3))
    ax_NE = fig_NE.add_subplot(1,1,1)
    offset_NE = math.floor(math.log10(np.max(df_data['err_total'])))
    max_NE_total = []
    
    for case_name, df_case in df_data.groupby('case'):
        NE_total = df_case['err_total'].values
        
        # mean_barh = np.array([np.mean(get_df[x]) for x in ['NE_stream', 'NE_g', 'NE_abs', 'NE_ins']])
        # check_barh = np.array(list(zip(['NE_stream', 'NE_g', 'NE_abs', 'NE_ins'],
        #                                [x / np.sum(mean_barh) for x in mean_barh])))
        # check_barh = check_barh[np.argsort(check_barh.T[1])]
        
        # iqr_NE_total = np.quantile(NE_total, 0.75) - np.quantile(NE_total, 0.25)
        # start_barh = np.quantile(NE_total, 0.25) - 1.5 * iqr_NE_total
        # end_barh = np.quantile(NE_total, 0.75) + 1.5 * iqr_NE_total
        
        # range_max_barh = float(check_barh[-1][1]) * (np.max(NE_total) - np.min(NE_total))
        # range_else_barh = (1 - float(check_barh[-1][1])) * (np.max(NE_total) - np.min(NE_total))
        
        # ax_NE.scatter([case_name], [np.max(NE_total) * pow(10, -offset_NE)], **pointstyle1_dict)
        ax_NE.boxplot(NE_total * pow(10, -offset_NE), positions=[case_name], showfliers=False, showmeans=True)
        # ax_NE.bar(x=it1+0.35, height=range_else_barh, bottom=np.min(NE_total)+range_max_barh, width=0.25,
        #           facecolor='None', edgecolor='black', linestyle='dashed')
        # ax_NE.annotate('{:n}%'.format(round(float(check_barh[-1][1]) * 100, 2)),
        #                xy=(it1+0.35, np.min(NE_total) + range_max_barh/2), xycoords='data',
        #                horizontalalignment='center', verticalalignment='center', rotation=0, fontsize=11)
        # ax_NE.annotate('{:n}%'.format(round(float(check_barh[-1][1]) * 100, 2)),
        #                xy=(it1+0.35, np.min(NE_total) + range_max_barh + 0.15), xycoords='data',
        #                horizontalalignment='center', verticalalignment='center', rotation=0, fontsize=11)
        
        max_NE_total.append(np.mean(NE_total) * pow(10, -offset_NE))
    
    id_max = np.where(max_NE_total >= np.max(max_NE_total))[0][0]
    id_min = np.where(max_NE_total <= np.min(max_NE_total))[0][0]
    range_ylim = ax_NE.get_ylim(); range_ylim = range_ylim[1] - range_ylim[0]
    ax_NE.set_ylim([ax_NE.get_ylim()[0] - 0.25 * range_ylim, ax_NE.get_ylim()[1] + 0.1 * range_ylim])
#     ax_NE.set_xlim([ax_NE.get_xlim()[0], ax_NE.get_xlim()[1] + 0.25])
    range_ylim = ax_NE.get_ylim()[1] - ax_NE.get_ylim()[0]
    
    ax_NE.hlines(max_NE_total[id_max], ax_NE.get_xlim()[0], ax_NE.get_xlim()[1],
                 color='red', linestyle='dotted')
    ax_NE.hlines(max_NE_total[id_min], ax_NE.get_xlim()[0], ax_NE.get_xlim()[1],
                 color='blue', linestyle='dotted')
    
    # for it1 in range(1,10):
    #     ax_NE.annotate(f'({dict_rename[check_barh[-1][0]]})',
    #                    xy=(it1+0.35, min_NE_total[it1-1] - 0.1 * range_ylim), xycoords='data',
    #                    horizontalalignment='center', verticalalignment='center', fontsize=11)
    # offset me
    ax_NE.text(0.02, 1.1, r'% ' + r'$\times 10^{' + f'{offset_NE}' + r'}$',
               horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize, transform=ax_NE.transAxes)
    
    # ax_NE.set_title('Galat neraca energi', pad=15)
    ax_NE.set_xlabel('Kasus')
    ax_NE.set_ylabel(r'$\Delta Q \, / \, Q_{tot}$', verticalalignment='center',
                       fontsize=y_offset_fontsize, labelpad=12.5)
    ax_NE.ticklabel_format(style='sci',scilimits=(-2,0),useMathText=False,useOffset=False,axis='y')
    ax_NE.set_xticks(list(range(1,10)))
    ax_NE2 = ax_NE.twinx()
    ax_NE2.set_yticks([round(max_NE_total[id_max], 2), round(max_NE_total[id_min], 2)])
    
    ax_NE2.set_ylim(ax_NE.get_ylim())
    
    # empirical mdot errors
    fig_type2 = plt.figure(figsize=(3.75,3))
    ax_type2 = fig_type2.add_subplot(1,1,1)

    df_data['err_type2'] = [np.abs(x) for x in df_data['err_type2']]
    
    offset_type2 = math.floor(math.log10(np.abs(np.max(df_data['err_type2']))))
    check_max_type2 = []
    
    for case_name, df_case in df_data.groupby('case'):
        check_max_type2.append(np.mean(df_case['err_type2'].values) * pow(10, -offset_type2))
        # ax_type2.scatter([case_name], [np.max(df_case['err_type2'].values) * pow(10, -offset_type2)], **pointstyle2_dict)
        ax_type2.boxplot(df_case['err_type2'].values * pow(10, -offset_type2), positions=[case_name], showfliers=False, showmeans=True)
        # ax_type2.boxplot(get_df['err_type2'].values * pow(10, -offset_type2), widths=0.3, positions=[it1], labels=[it1])
    
    max_type2 = check_max_type2[np.where(check_max_type2 >= np.max(check_max_type2))[0][0]]
    min_type2 = check_max_type2[np.where(check_max_type2 <= np.min(check_max_type2))[0][0]]
    
    ax_type2.hlines(max_type2, ax_type2.get_xlim()[0], ax_type2.get_xlim()[1],
                    color='red', linestyle='dotted')
    ax_type2.hlines(min_type2, ax_type2.get_xlim()[0], ax_type2.get_xlim()[1],
                    color='blue', linestyle='dotted')
    
    # ax_type2.set_title('Tipe II', pad=15)
    # offset me
    ax_type2.text(0.02, 1.1, r'% ' + r'$\times 10^{' + f'{offset_type2}' + r'}$',
                  horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize,
                  transform=ax_type2.transAxes)
    
    range_ylim_type2 = ax_type2.get_ylim()[1] - ax_type2.get_ylim()[0]
    
    ax_type2.set_ylabel(r'$\Delta \dot{m} \, / \, \dot{m}_{II}$', verticalalignment='center',
                        size = y_offset_fontsize, labelpad=12.5)
    ax_type2.set_xlabel('Kasus')
    ax_type2.ticklabel_format(style='sci',scilimits=(-2,0),useMathText=False,useOffset=False,axis='y')
    
    ax_type2.set_xticks(list(range(1,10)))

    ax_type22 = ax_type2.twinx()
    ax_type22.set_yticks([round(max_type2, 2), round(min_type2, 2)])
    
    ax_type22.set_ylim(ax_type2.get_ylim())

    # rmsr and CFL
    for it1, it2 in zip(['u', 'v', 'w', 'T', 'k', 'omega'], [r'\mathscr{u}', r'\mathscr{v}', r'\mathscr{w}', r'T', r'k', r'\omega']):
        fig_log = plt.figure(figsize=(3.75,3))
        ax_rmsr = host_subplot(111, axes_class = AA.Axes)
        fig_log.add_subplot(ax_rmsr)
        ax_cfl = ax_rmsr.twinx()
        ax_cfl2 = ax_rmsr.twinx()
        
        new_fixed_axis = ax_cfl2.get_grid_helper().new_fixed_axis
        ax_cfl2.axis["right"] = new_fixed_axis(loc="right", axes=ax_cfl2,
                                                offset=(30, 0))
        ax_cfl2.axis["right"].toggle(all=True)
        
        try:
            offset_rmsr = math.floor(math.log10(np.max(df_data[f'rmsr_{it1}'].values)))
        except:
            offset_rmsr = 0
        try:
            offset_cfl = math.floor(math.log10(np.max(df_data[f'cfl_{it1}'].values)))
        except:
            offset_cfl = 0
        
        cfl_ticks = []
        
        offset_iter = math.floor(math.log10(np.max(df_data['times'].values)))

        for case_name, df_case in df_data.groupby('case'):
            ax_rmsr.plot(df_case['times'] * pow(10, -offset_iter), df_case[f'rmsr_{it1}'] * pow(10, -offset_rmsr), color='black', linestyle='dotted')
            cfl_ticks.append(np.max(df_case[f'cfl_{it1}']) * pow(10, -offset_cfl))
        
        # ax_cfl.set_ylim([np.max(cfl_ticks), np.min(cfl_ticks)])
        # ax_cfl2.set_ylim(ax_cfl.get_ylim())
        range_cfl_ticks = np.max(cfl_ticks) - np.min(cfl_ticks)

        for case_name, cfl_case in enumerate(cfl_ticks):
            circ_name = ' '
            if cfl_case >= np.max(cfl_ticks):
                circ_name = case_name
            try:
                ax_cfl2.annotate(f' ', xy=(1.05, (cfl_case - np.min(cfl_ticks))/range_cfl_ticks), xycoords='axes fraction',
                                horizontalalignment='center', verticalalignment='center',
                                bbox = dict(boxstyle=f"circle,pad={0.005}", fc="None", ec="black"), fontsize=8)
                ax_cfl2.annotte(f'{case_name}', xy=(1.05, (cfl_case - np.min(cfl_ticks))/range_cfl_ticks + 0.01), xycoords='axes fraction',
                                horizontalalignment='center', verticalalignment='center',
                                bbox = dict(boxstyle=f"circle,pad={0.005}", fc="None", ec="black"), fontsize=8)
            except:
                ax_cfl2.annotate(' ', xy=(1.05, 0.5), xycoords='axes fraction',
                                horizontalalignment='center', verticalalignment='center',
                                bbox = dict(boxstyle=f"circle,pad={0.005}", fc="None", ec="black"), fontsize=8)

        check_max_rmsr = np.max(df_data[f'rmsr_{it1}'].values) * pow(10, -offset_rmsr)
        check_max_rmsr_id = df_data[df_data[f'rmsr_{it1}'] >= np.max(df_data[f'rmsr_{it1}'].values)]['case'].values[0]
        check_max_cfl = np.max(cfl_ticks)
        
        hline_rmsr = ax_rmsr.plot(ax_rmsr.get_xlim(), [check_max_rmsr, check_max_rmsr], color='red', linestyle='dashed', label=check_max_rmsr_id)
        labelLines(hline_rmsr, align=True)
        
        ax_rmsr.set_xlabel('Waktu ' + r'$[s]$', labelpad=10, horizontalalignment='center')
        ax_rmsr.set_ylabel('RMSR ' + r'$[' + it2 + r']$' + ' ({:n})'.format(round(check_max_rmsr, 2)), labelpad=25,
                           verticalalignment='center')
        ax_cfl2.set_ylabel('maks. CFL ' + r'$[' + it2 + r']$' + ' ({:n})'.format(round(check_max_cfl, 2)), labelpad=55,
                          verticalalignment='center')
        ax_cfl.set_ylim([np.min(cfl_ticks), np.max(cfl_ticks)]); ax_cfl2.set_ylim([np.min(cfl_ticks), np.max(cfl_ticks)])
        ax_cfl2.set_yticks([round(x,2) for x in [np.min(cfl_ticks), np.mean(cfl_ticks), np.max(cfl_ticks)]])
        # offset me
        ax_rmsr.text(0.02, 1.1, r'$\times 10^{' + f'{offset_rmsr}' + r'}$',
                    horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize,
                    transform=ax_rmsr.transAxes)
        ax_rmsr.text(0.98, 1.1, r'$\times 10^{' + f'{offset_cfl}' + r'}$',
                    horizontalalignment='right', verticalalignment='top', size = y_offset_fontsize,
                    transform=ax_rmsr.transAxes)
        ax_rmsr.text(0.98, -0.2, r'$\times 10^{' + f'{offset_iter}' + r'}$',
                     horizontalalignment='right', verticalalignment='bottom', size = y_offset_fontsize,
                    transform=ax_rmsr.transAxes)
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\galat'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\galat', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\galat')
            # print('Plotting results...')
            fig_log.savefig(f'log_{it1}_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print(f'Saved to log_{it1}_plot.pdf')
            plt.close(fig_log)
        
    if save_ is True:
        if not os.path.exists(f'{caseloc}\\plot\\galat'):
            try:
                original_umask = os.umask(0)
                os.makedirs(f'{caseloc}\\plot\\galat', 0o777)
            finally:
                os.umask(original_umask)
        os.chdir(f'{caseloc}\\plot\\galat')
        # print('Plotting results...')
        fig_NE.savefig(f'NE_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
        fig_type2.savefig(f'type2_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
        print('Saved to x_plot.pdf')
        plt.close(fig_NE)
        plt.close(fig_type2)

# plot compounds
def plot_compound(df_smooth_data, save_: bool = True):
    # vs time, radial dist plot
    # v_stream, cf, T_fluid, h, Nu, Gr, Re, Ri
    # [v_stream / GrCf] vs 1 - exp(-NuFo), eta vs 1 - exp(-NuFo) <- vs time only
    
    vars_l = ['v_stream', 'Cf', 'Pgrady_stream', 'T_fluid', 'T_surface', 'h', 'Nu', 'Gr', 'Re', 'Ri',
              'Rt', 'ACH', 'psi', 'eta', 'reg_v_y', 'reg_eta_y', 'NuFo']
    
    df_compound = deepcopy(df_smooth_data)
    
    df_compound['v_stream'] = df_compound['v_stream'] - v0_
    df_compound['T_fluid'] = df_compound['T_fluid'] - T0_
    df_compound['T_surface'] = df_compound['T_surface'] - T0_
    df_compound['Nu'] = df_compound['Nu'] - 1
    
    df_compound['Ri'] = [x/pow(y,2) for x,y in zip(df_compound['Gr'].values, df_compound['Re'].values)]
    
    # identify max-min, center (9)
    get_dist = {}
    ref_case = deepcopy(df_compound[df_compound['case'] == 9])
    for it1 in vars_l:
        get_dist[it1] = []
        for case_name, df_case in df_compound.groupby('case'):
            get_dist[it1].append(np.mean(df_case[it1].values - ref_case[it1].values))
    get_max = dict(zip(vars_l, [np.where(x >= np.max(x))[0][0] for x in list(get_dist.values())]))
    get_min = dict(zip(vars_l, [np.where(x <= np.min(x))[0][0] for x in list(get_dist.values())]))
    
    # prep radial plot
    sort_doe = [4,3,6,1,5,0,7,2]
    angle_doe = np.linspace(0,2*np.pi,9,endpoint=True)

    loc_doe = np.array([[scos(x), ssin(x)] for x in angle_doe]); loc_doe = loc_doe.T

    channel_H_trans = channel_H[sort_doe]
    channel_W_trans = channel_W[sort_doe]
    
    times_ = df_compound[df_compound['case'] == 1]['times'].values
    
    vars_nounit = [r'$(\overline{\mathscr{v}} - \mathscr{v}_{0})$' + ' ' + r'$[\mathrm{m}\mathrm{s}^{-1}]$',
                   r'$C_{f}$',
                   r'$\partial P / \partial y$' + ' ' + r'$[\mathrm{Pa}$' + ' ' r'$\mathrm{m}^{-1}]$',
                   r'$(\overline{T} - T_{0})$' + ' ' + r'$[\mathrm{K}]$',
                   r'$(\overline{T_{s}} - T_{0})$' + ' ' + r'$[\mathrm{K}]$',
                   r'$h$' + ' ' + r'$[\mathrm{W}\mathrm{m}^{-2}\mathrm{K}^{-1}]$',
                   r'$(Nu - 1)$',
                   r'$Gr$',
                   r'$Re$',
                   r'$Ri$',
                   r'$R_{t}$',
                   r'ACH',
                   r'$\mathscr{v}^{*}$',
                   r'$\eta$',
                   r'$\mathscr{v}^{*} \sim NuFo$',
                   r'$\eta \sim NuFo$',
                   r'$NuFo$',
                   ]
    vars_unit = [r'$\left . \overline{\Delta} (\overline{\mathscr{v}} - \mathscr{v}_{0}) \right |_{9}$',
                r'$\left . \overline{\Delta} C_{f} \right |_{9}$',
                r'$\left . \overline{\Delta} \partial P / \partial y \right |_{9}$',
                r'$\left . \overline{\Delta} (\overline{T} - T_{0}) \right |_{9}$',
                r'$\left . \overline{\Delta} (\overline{T_{s}} - T_{0}) \right |_{9}$',
                r'$\left . \overline{\Delta} h \right |_{9}$',
                r'$\left . \overline{\Delta} (Nu - 1) \right |_{9}$',
                r'$\left . \overline{\Delta} Gr \right |_{9}$',
                r'$\left . \overline{\Delta} Re \right |_{9}$',
                r'$\left . \overline{\Delta} Ri \right |_{9}$',
                r'$\left . \overline{\Delta} R_{t} \right |_{9}$',
                r'$\left . \overline{\Delta} \mathrm{ACH} \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}^{*} \right |_{9}$',
                r'$\left . \overline{\Delta} \eta \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}^{*} \sim NuFo \right |_{9}$',
                r'$\left . \overline{\Delta} \eta \sim NuFo \right |_{9}$',
                r'$\left . \overline{\Delta} NuFo \right |_{9}$'
                ]
    vars_rad = [r'$\overline{\mathscr{v}}$',
                r'$C_{f}$',
                r'$\partial P / \partial y$',
                r'$(\overline{T} - T_{0})$',
                r'$(\overline{T}_{s} - T_{0})$',
                r'$h$',
                r'$(Nu - 1)$',
                r'$Gr$',
                r'$Re$',
                r'$Ri$',
                r'$R_{t}$',
                r'ACH',
                r'$\mathscr{v}^{*}$',
                r'$\eta$',
                r'$\mathscr{v}^{*} \sim NuFo$',
                r'$\eta \sim NuFo$',
                r'$NuFo$']
    vars_y = [[['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']],
                [['NuFo', r'$NuFo$'], ['times', 'Waktu [s]']],
                [['NuFo', r'$NuFo$'], ['times', 'Waktu [s]']],
                [['NuFo', r'$NuFo$'], ['times', 'Waktu [s]']],
                [['NuFo', r'$NuFo$'], ['times', 'Waktu [s]']],
                [['times', 'Waktu [s]']]]
    
    vars_nounit = dict(zip(vars_l, vars_nounit))
    vars_unit = dict(zip(vars_l, vars_unit))
    vars_offset = dict(zip(vars_l, [math.floor(math.log10(np.max([np.abs(x) for x in df_compound[x].values]))) for x in vars_l]))
    nufo_offset = math.floor(math.log10(np.max([np.abs(x) for x in df_compound['NuFo'].values])))
    vars_rad = dict(zip(vars_l, vars_rad))
    
    # plot vs time and radial dist plot
    vars_norm = dict.fromkeys(vars_l, np.zeros(shape=(2,), dtype=float))
    
    for var_id, vars_ in enumerate(vars_l):
        # vs time
        for time_id, time_this in enumerate(vars_y[var_id]):
            fig_vs_time = plt.figure(figsize=(3.75,3))
            ax_vs_time = fig_vs_time.add_subplot(1,1,1)
            
            get_else = list(range(0,9)); get_else.remove(get_max[vars_]); get_else.remove(get_min[vars_])
            
            if time_this[0] in ['NuFo']:
                get_y = df_compound[df_compound['case'] == get_max[vars_]+1]['NuFo'].values * pow(10, -nufo_offset)
            else:
                get_y = df_compound[df_compound['case'] == get_max[vars_]+1]['times'].values
            
            ax_vs_time.plot(get_y, df_compound[df_compound['case'] == get_max[vars_]+1][vars_].values * pow(10, -vars_offset[vars_]),
                            color='red', linestyle='solid', label=r'$\overline{' + f'{get_max[vars_]+1}' r'}$')
            ax_vs_time.plot(get_y, df_compound[df_compound['case'] == get_min[vars_]+1][vars_].values * pow(10, -vars_offset[vars_]),
                            color='blue', linestyle='solid', label=get_min[vars_]+1)
            if any([8 == x for x in get_else]) is True:
                ax_vs_time.plot(get_y, df_compound[df_compound['case'] == 9][vars_].values * pow(10, -vars_offset[vars_]),
                                color='black', linestyle='dashed', label=9)
                get_else.remove(8)
            # offset me
            ax_vs_time.text(0.02, 1.1, r'$\times 10^{' + f'{vars_offset[vars_]}' + r'}$',
                    horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize,
                    transform=ax_vs_time.transAxes)
            
            if time_this[0] in ['NuFo']:
                ax_vs_time.text(0.98, -0.2, r'$\times 10^{' + f'{nufo_offset}' + r'}$',
                        horizontalalignment='right', verticalalignment='center',
                        transform=ax_vs_time.transAxes)
                
            labelLines(ax_vs_time.get_lines(), align=True)
            
            for else_id in get_else:
                ax_vs_time.plot(get_y, df_compound[df_compound['case'] == else_id+1][vars_].values * pow(10, -vars_offset[vars_]),
                                color='black', linestyle='dotted', alpha=0.6)
                        
            ax_vs_time.set_xlabel(time_this[1], labelpad=10, horizontalalignment='center')
            ax_vs_time.set_ylabel(vars_nounit[vars_], labelpad=12.5, size = y_offset_fontsize, verticalalignment='center')
            ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])
            
            ax_vs_time2 = ax_vs_time.twinx()
            ax_vs_time2.set_yticks([round(df_compound[df_compound['case'] == 9][vars_].values[-1] * pow(10, -vars_offset[vars_]),2)])
            ax_vs_time2.set_ylim(ax_vs_time.get_ylim())
            
            if save_ is True:
                if not os.path.exists(f'{caseloc}\\plot\\explore'):
                    try:
                        original_umask = os.umask(0)
                        os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
                    finally:
                        os.umask(original_umask)
                os.chdir(f'{caseloc}\\plot\\explore')
                # print('Plotting results...')
                fig_vs_time.savefig(f'{vars_}_{time_this_trans[time_this[0]]}_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
                print('Saved to x_plot.pdf')
                plt.close(fig_vs_time)
            
        # radial plot
        fig_radial = plt.figure(figsize=(3,3))
        ax_radial = fig_radial.add_subplot(1,1,1)
        
        loc_doe_x = np.array([channel_H_transform(x) for x in loc_doe[0]])
        loc_doe_y = np.array([channel_W_transform(x) for x in loc_doe[1]])
        
        get_dist_temp = [get_dist[vars_][x] for x in sort_doe]
        
        max_obv = np.max([np.abs(x) for x in get_dist_temp])        
        offset_multip_obv = math.floor(math.log10(max_obv))
        
        radius_obv_doe = np.array([np.abs(x) / max_obv for x in get_dist_temp])
        
        loc_obv_doe = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, radius_obv_doe)]); loc_obv_doe = loc_obv_doe.T
        loc_obv_doe_vec = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, get_dist_temp / max_obv)]); loc_obv_doe_vec = loc_obv_doe_vec.T
        
        avg_vec = np.array([np.sum(loc_obv_doe_vec[0]), np.sum(loc_obv_doe_vec[1])], dtype='float')
        avg_vec = avg_vec.astype('float')
        avg_vec = np.array([np.dot(avg_vec, np.array([1,0])) / np.linalg.norm(avg_vec),
                            np.dot(avg_vec, np.array([0,1])) / np.linalg.norm(avg_vec)])
        vars_norm[vars_] = avg_vec
        
        loc_obv_doe_x = np.array([channel_H_transform(x) for x in loc_obv_doe[0]])
        loc_obv_doe_y = np.array([channel_W_transform(x) for x in loc_obv_doe[1]])
        
        ax_radial.scatter(loc_doe_x, loc_doe_y, marker='s', **pointstyle2_dict)
        ax_radial.scatter(loc_obv_doe_x, loc_obv_doe_y, marker='s', **pointstyle1_dict)

        for it1, it2 in zip(loc_obv_doe_x, loc_obv_doe_y):
            ax_radial.plot([mean_H, it1], [mean_W, it2], color='black', linestyle='solid', linewidth=1)
        
        ax_radial.annotate('', xy=(round(channel_H_transform(avg_vec[0]), 2), round(channel_W_transform(avg_vec[1]), 2)), xytext=(channel_H_transform(0), channel_W_transform(0)),
                           xycoords='data', arrowprops=dict(arrowstyle='->', linestyle='dashed', color='red', alpha=1))
        ax_radial.annotate('< {:n}'.format(round(avg_vec[0],2)) + r'$\hat{i}$' + ' + {:n}'.format(round(avg_vec[1],2)) + r'$\hat{j}$' + ' >', xy=(0.98,0.94), xycoords='axes fraction',
                            horizontalalignment='right', verticalalignment='center',
                            bbox = dict(boxstyle=f"square,pad={0.15}", fc="None", ec="black", linewidth=0.5), fontsize=10)
        
        for it1, it2, it3, it4 in zip(sort_doe, loc_doe[0], loc_doe[1], get_dist_temp):
            ax_radial.annotate('{:.0f}'.format(it1+1) + '\n' + '({:n})'.format(round(it4 * pow(10, -offset_multip_obv),2)),
                            xy=(channel_H_transform(it2 * 1.35), channel_W_transform(it3 * 1.35)), xycoords='data',
                            horizontalalignment='center', verticalalignment='center',fontsize=11)    
        
        range_xlim = ax_radial.get_xlim()[1] - ax_radial.get_xlim()[0]
        range_ylim = ax_radial.get_ylim()[1] - ax_radial.get_ylim()[0]
        new_xlim = [x + y for x,y in zip(ax_radial.get_xlim(), [-range_xlim*0.3, range_xlim*0.3])]
        new_ylim = [x + y for x,y in zip(ax_radial.get_ylim(), [-range_ylim*0.3, range_ylim*0.3])]
        ax_radial.set_xlim(new_xlim); ax_radial.set_ylim(new_ylim)
        
        ax_radial.set_title(f'{vars_unit[vars_]} ' + r'$\times 10^{' + f'{offset_multip_obv}' + r'}$', pad=15)
        
        ax_radial.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_radial.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5, verticalalignment='center')
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\explore'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\explore')
            # print('Plotting results...')
            fig_radial.savefig(f'{vars_}_dist_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_plot.pdf')
            plt.close(fig_radial)

    # plot all radial dist and calc vector corr.
    fig_radial_all = plt.figure(figsize=(3,3))
    ax_radial_all = fig_radial_all.add_subplot(1,1,1)
    
    loc_doe_x = np.array([channel_H_transform(x) for x in loc_doe[0]])
    loc_doe_y = np.array([channel_W_transform(x) for x in loc_doe[1]])
    
    ax_radial_all.scatter(loc_doe_x, loc_doe_y, marker='s', **pointstyle2_dict)

    eval_maxes = np.array(list(vars_norm.values())).T
    max_to_x = list(vars_norm.keys())[np.where(eval_maxes[0] >= np.max(eval_maxes[0]))[0][0]]
    min_to_x = list(vars_norm.keys())[np.where(eval_maxes[0] <= np.min(eval_maxes[0]))[0][0]]
    max_to_y = list(vars_norm.keys())[np.where(eval_maxes[1] >= np.max(eval_maxes[1]))[0][0]]
    min_to_y = list(vars_norm.keys())[np.where(eval_maxes[1] <= np.min(eval_maxes[1]))[0][0]]
    
    if_mark = [max_to_x, max_to_y, min_to_x, min_to_y]
    else_mark = list(vars_norm.keys()); [else_mark.remove(x) for x in list(vars_norm.keys()) if x in if_mark]
    
    for it1 in if_mark:
        it2 = vars_norm[it1]
        ax_radial_all.annotate('', xy=(round(channel_H_transform(it2[0]), 2), round(channel_W_transform(it2[1]), 2)),
                               xytext=(channel_H_transform(0), channel_W_transform(0)), xycoords='data',
                               arrowprops=dict(arrowstyle='->', linestyle='solid', color='black', alpha=1))
        ax_radial_all.scatter([round(channel_H_transform(it2[0]), 2)], [round(channel_W_transform(it2[1]), 2)], **pointstyle1_dict)
        ax_radial_all.annotate(f'{vars_rad[it1]}', xy=(channel_H_transform(it2[0] * 1.25), channel_W_transform(it2[1] * 1.25)), xycoords='data',
                               horizontalalignment='center', verticalalignment='center',fontsize=11) 
    
    for it1 in else_mark:
        it2 = vars_norm[it1]
        ax_radial_all.annotate('', xy=(round(channel_H_transform(it2[0]), 2), round(channel_W_transform(it2[1]), 2)),
                               xytext=(channel_H_transform(0), channel_W_transform(0)), xycoords='data',
                               arrowprops=dict(arrowstyle='->', linestyle='dashed', color='black', alpha=0.6))
    
    range_xlim = ax_radial_all.get_xlim()[1] - ax_radial_all.get_xlim()[0]
    range_ylim = ax_radial_all.get_ylim()[1] - ax_radial_all.get_ylim()[0]
    new_xlim = [x + y for x,y in zip(ax_radial_all.get_xlim(), [-range_xlim*0.2, range_xlim*0.2])]
    new_ylim = [x + y for x,y in zip(ax_radial_all.get_ylim(), [-range_ylim*0.2, range_ylim*0.2])]
    ax_radial_all.set_xlim(new_xlim); ax_radial_all.set_ylim(new_ylim)
    
    # weak-strong corr lines
    # exclude_hehe = 'Gr'
    # check_cor_lims = np.array([y for x,y in vars_norm.items() if all([x not in [exclude_hehe]])]).T
    # check_pos_x_cor_lims = np.array([x for x in check_cor_lims[0] if all([x>=0])]) * 0.9
    # check_neg_x_cor_lims = np.array([x for x in check_cor_lims[0] if all([x<0])]) * 0.9
    # check_pos_y_cor_lims = np.array([x for x in check_cor_lims[1] if all([x>=0])]) * 0.9
    
    ax_radial_all2 = ax_radial_all.twinx()
    ax_radial_all2.plot(ax_radial_all.get_xlim(),
                        [channel_W_transform(0.2), channel_W_transform(0.2)],
                        linestyle='dotted', color='red', linewidth=1, alpha=0.5)
    ax_radial_all2.plot(ax_radial_all.get_xlim(),
                        [channel_W_transform(-0.2), channel_W_transform(-0.2)],
                        linestyle='dotted', color='red', linewidth=1, alpha=0.5)
    ax_radial_all2.plot([channel_H_transform(-0.8), channel_H_transform(-0.8)],
                        ax_radial_all.get_ylim(), linestyle='dotted', color='red', linewidth=1, alpha=0.5)
    ax_radial_all2.plot([channel_H_transform(0.8), channel_H_transform(0.8)],
                        ax_radial_all.get_ylim(), linestyle='dotted', color='red', linewidth=1, alpha=0.5)

    ax_radial_all.annotate(r'$r_{H} = \pm$' + ' 0,8' + '\n' + r'$r_{W} = \pm$' + ' 0,2', xy=(0.05,0.9), xycoords='axes fraction',
                        horizontalalignment='left', verticalalignment='center',
                        bbox = dict(boxstyle=f"square,pad={0.3}", fc="None", ec="red", linewidth=0.5), color='red', fontsize=10)
    ax_radial_all.annotate(r'$\pm$' + ' 0,8', xy=(0.83, 0.9), xycoords='axes fraction',
                           horizontalalignment='center', verticalalignment='center',
                           color='red', fontsize=9)
    ax_radial_all.annotate(r'$\pm$' + ' 0,2', xy=(0.1, 0.4), xycoords='axes fraction',
                           horizontalalignment='center', verticalalignment='center',
                           color='red', fontsize=9)
    
    ax_radial_all.set_title(r'$\hat{n}$' + ' ' + r'$\left . \Delta \phi \right |_{9}$', pad=15)
    
    ax_radial_all.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
    ax_radial_all.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5, verticalalignment='center')    

    if save_ is True:
        if not os.path.exists(f'{caseloc}\\plot\\explore'):
            try:
                original_umask = os.umask(0)
                os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
            finally:
                os.umask(original_umask)
        os.chdir(f'{caseloc}\\plot\\explore')
        # print('Plotting results...')
        fig_radial_all.savefig(f'all_dist_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
        print('Saved to x_plot.pdf')
        plt.close(fig_radial_all)
    
    cors_norm = pd.DataFrame(columns=list(vars_norm.keys()), index=list(vars_norm.keys()))
    for it1 in list(vars_norm.keys()):
        for it2 in list(vars_norm.keys()):
            cors_norm.at[it1, it2] = np.dot(vars_norm[it1], vars_norm[it2]) / \
                (np.linalg.norm(vars_norm[it1]) * np.linalg.norm(vars_norm[it2]))
    
    os.chdir(f'{caseloc}\\tabel')
    cors_norm.to_csv(f'cors_norm_{casename}.csv')

def plot_compound_model(df_smooth_data, save_: bool = True):
    # vs time, radial dist plot
    # v_stream, cf, T_fluid, h, Nu, Gr, Re, Ri
    # [v_stream / GrCf] vs 1 - exp(-NuFo), eta vs 1 - exp(-NuFo) <- vs time only
    
    vars_l = ['psi_model', 'eta_model', 'psi_kurva', 'eta_kurva',
              'v_stream_model', 'ACH_model', 'Rt_model',
              'T_fluid_model', 'h_eta_model', 'h_psi_model', 'Nu_eta_model', 'Nu_psi_model', 'q_model', 'v_dn_model']
    vars_annotate = ['Model', 'Model', 'Kurva', 'Kurva',
                     'Model', 'Model', 'Model', 'Model', 'Model', 'Model', 'Model', 'Model', 'Model', 'Model']
    
    df_compound = deepcopy(df_smooth_data)
    df_compound['v_stream_model'] = df_compound['v_stream_model'].values - v0_
    df_compound['T_fluid_model'] = df_compound['T_fluid_model'].values - T0_
    
    # identify max-min, center (9)
    get_dist = {}
    ref_case = deepcopy(df_compound[df_compound['case'] == 9])
    for it1 in vars_l:
        get_dist[it1] = []
        for case_name, df_case in df_compound.groupby('case'):
            get_dist[it1].append(np.mean(df_case[it1].values - ref_case[it1].values))
    get_max = dict(zip(vars_l, [np.where(x >= np.max(x))[0][0] for x in list(get_dist.values())]))
    get_min = dict(zip(vars_l, [np.where(x <= np.min(x))[0][0] for x in list(get_dist.values())]))
    
    # prep radial plot
    sort_doe = [4,3,6,1,5,0,7,2]
    angle_doe = np.linspace(0,2*np.pi,9,endpoint=True)

    loc_doe = np.array([[scos(x), ssin(x)] for x in angle_doe]); loc_doe = loc_doe.T

    channel_H_trans = channel_H[sort_doe]
    channel_W_trans = channel_W[sort_doe]
    
    times_ = df_compound[df_compound['case'] == 1]['times'].values
    
    vars_nounit = [r'$\mathscr{v}^{*}$',
                   r'$\eta$',
                   r'$\mathscr{v}^{*}$',
                   r'$\eta$',
                   r'$(\overline{\mathscr{v}} - \mathscr{v}_{0})$' + ' ' + r'$[\mathrm{m}\mathrm{s}^{-1}]$',
                   r'ACH',
                   r'$R_{t}$',
                   r'$(\overline{T} - T_{0})$' + ' ' + '[K]',
                   r'$h \, [\eta]$' + ' ' + r'$[\mathrm{W}\mathrm{m}^{-2}\mathrm{K}^{-1}]$',
                   r'$h \, [\mathscr{v}^{*}]$' + ' ' + r'$[\mathrm{W}\mathrm{m}^{-2}\mathrm{K}^{-1}]$',
                   r'$Nu \, [\eta]$',
                   r'$Nu \, [\mathscr{v}^{*}]$',
                   r'$\dot{Q}_{eff}$' + ' ' + '[W]',
                   r'$\mathscr{v}_{DN, \, eff}$' + ' ' + r'$[ms^{-1}]$'
                   ]
    vars_unit = [r'$\left . \overline{\Delta} \mathscr{v}^{*} \right |_{9}$',
                r'$\left . \overline{\Delta} \eta \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}^{*} \right |_{9}$',
                r'$\left . \overline{\Delta} \eta \right |_{9}$',
                r'$\left . \overline{\Delta} (\overline{\mathscr{v}} - \mathscr{v}_{0}) \right |_{9}$',
                r'$\left . \overline{\Delta} \mathrm{ACH} \right |_{9}$',
                r'$\left . \overline{\Delta} R_{t} \right |_{9}$',
                r'$\left . \overline{\Delta} (\overline{T} - T_{0}) \right |_{9}$',
                r'$\left . \overline{\Delta} h \, [\eta] \right |_{9}$',
                r'$\left . \overline{\Delta} h \, [\mathscr{v}^{*}] \right |_{9}$',
                r'$\left . \overline{\Delta} Nu \, [\eta] \right |_{9}$',
                r'$\left . \overline{\Delta} Nu \, [\mathscr{v}^{*}] \right |_{9}$',
                r'$\left . \overline{\Delta} \dot{Q}_{eff} \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}_{DN, \, eff} \right |_{9}$'
                ]
    vars_rad = [r'$\mathscr{v}^{*}$',
                r'$\eta$',
                r'$\mathscr{v}^{*}$',
                r'$\eta$',
                r'$\overline{\mathscr{v}}$',
                r'ACH',
                r'$R_{t}$',
                r'$(\overline{T} - T_{0})$',
                r'$h \, [\eta]$',
                r'$h \, [\mathscr{v}^{*}]$',
                r'$Nu \, [\eta]$',
                r'$Nu \, [\mathscr{v}^{*}]$',
                r'$\dot{Q}_{eff}$',
                r'$\mathscr{v}_{DN, \, eff}$'
                ]
    
    vars_nounit = dict(zip(vars_l, vars_nounit))
    vars_unit = dict(zip(vars_l, vars_unit))
    vars_offset = {}
    for it1 in vars_l:
        vars_offset[it1] = math.floor(math.log10(np.max([np.abs(x) for x in df_compound[it1].values])))
    vars_rad = dict(zip(vars_l, vars_rad))
    
    # plot vs time and radial dist plot
    vars_norm = dict.fromkeys(vars_l, np.zeros(shape=(2,), dtype=float))
    
    for var_id, vars_ in enumerate(vars_l):
        # vs time
        fig_vs_time = plt.figure(figsize=(3.75,3))
        ax_vs_time = fig_vs_time.add_subplot(1,1,1)
        
        get_else = list(range(0,9)); get_else.remove(get_max[vars_])
        if get_max[vars_] != get_min[vars_]:
            get_else.remove(get_min[vars_])
        
        ax_vs_time.plot(times_, df_compound[df_compound['case'] == get_max[vars_]+1][vars_].values * pow(10, -vars_offset[vars_]),
                        color='red', linestyle='solid', label=r'$\overline{' + f'{get_max[vars_]+1}' r'}$')
        ax_vs_time.plot(times_, df_compound[df_compound['case'] == get_min[vars_]+1][vars_].values * pow(10, -vars_offset[vars_]),
                        color='blue', linestyle='solid', label=get_min[vars_]+1)
        if any([8 == x for x in get_else]) is True:
            ax_vs_time.plot(times_, df_compound[df_compound['case'] == 9][vars_].values * pow(10, -vars_offset[vars_]),
                            color='black', linestyle='dashed', label=9)
            get_else.remove(8)
        
        ax_vs_time.text(0.02, 1.1, r'$\times 10^{' + f'{vars_offset[vars_]}' + r'}$',
                horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize,
                transform=ax_vs_time.transAxes)
        ax_vs_time.text(0.98, 1.02, vars_annotate[var_id], horizontalalignment='right', verticalalignment='bottom', size=12,
                        transform=ax_vs_time.transAxes)
        
        labelLines(ax_vs_time.get_lines(), align=True)
        
        for else_id in get_else:
            ax_vs_time.plot(times_, df_compound[df_compound['case'] == else_id+1][vars_].values * pow(10, -vars_offset[vars_]),
                            color='black', linestyle='dotted', alpha=0.6)
        
        ax_vs_time.set_xlabel('Waktu [s]', labelpad=10, horizontalalignment='center')
        ax_vs_time.set_ylabel(vars_nounit[vars_], labelpad=12.5, size = y_offset_fontsize, verticalalignment='center')
        ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])

        ax_vs_time2 = ax_vs_time.twinx()
        ax_vs_time2.set_yticks([round(df_compound[df_compound['case'] == 9][vars_].values[-1] * pow(10, -vars_offset[vars_]),2)])
        ax_vs_time2.set_ylim(ax_vs_time.get_ylim())
        
        # radial plot
        fig_radial = plt.figure(figsize=(3,3))
        ax_radial = fig_radial.add_subplot(1,1,1)
        
        loc_doe_x = np.array([channel_H_transform(x) for x in loc_doe[0]])
        loc_doe_y = np.array([channel_W_transform(x) for x in loc_doe[1]])
        
        get_dist_temp = [get_dist[vars_][x] for x in sort_doe]
        
        max_obv = np.max([np.abs(x) for x in get_dist_temp])        
        offset_multip_obv = math.floor(math.log10(max_obv))
        
        radius_obv_doe = np.array([np.abs(x) / max_obv for x in get_dist_temp])
        
        loc_obv_doe = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, radius_obv_doe)]); loc_obv_doe = loc_obv_doe.T
        loc_obv_doe_vec = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, get_dist_temp / max_obv)]); loc_obv_doe_vec = loc_obv_doe_vec.T
        
        avg_vec = np.array([np.sum(loc_obv_doe_vec[0]), np.sum(loc_obv_doe_vec[1])], dtype='float')
        avg_vec = avg_vec.astype('float')
        avg_vec = np.array([np.dot(avg_vec, np.array([1,0])) / np.linalg.norm(avg_vec),
                            np.dot(avg_vec, np.array([0,1])) / np.linalg.norm(avg_vec)])
        vars_norm[vars_] = avg_vec
        
        loc_obv_doe_x = np.array([channel_H_transform(x) for x in loc_obv_doe[0]])
        loc_obv_doe_y = np.array([channel_W_transform(x) for x in loc_obv_doe[1]])
        
        ax_radial.scatter(loc_doe_x, loc_doe_y, marker='s', **pointstyle2_dict)
        ax_radial.scatter(loc_obv_doe_x, loc_obv_doe_y, marker='s', **pointstyle1_dict)

        for it1, it2 in zip(loc_obv_doe_x, loc_obv_doe_y):
            ax_radial.plot([mean_H, it1], [mean_W, it2], color='black', linestyle='solid', linewidth=1)
        
        ax_radial.annotate('', xy=(round(channel_H_transform(avg_vec[0]), 2), round(channel_W_transform(avg_vec[1]), 2)), xytext=(channel_H_transform(0), channel_W_transform(0)),
                           xycoords='data', arrowprops=dict(arrowstyle='->', linestyle='dashed', color='red', alpha=1))
        ax_radial.annotate('< {:n}'.format(round(avg_vec[0],2)) + r'$\hat{i}$' + ' + {:n}'.format(round(avg_vec[1],2)) + r'$\hat{j}$' + ' >', xy=(0.98,0.94), xycoords='axes fraction',
                            horizontalalignment='right', verticalalignment='center',
                            bbox = dict(boxstyle=f"square,pad={0.15}", fc="None", ec="black", linewidth=0.5), fontsize=10)
        
        for it1, it2, it3, it4 in zip(sort_doe, loc_doe[0], loc_doe[1], get_dist_temp):
            ax_radial.annotate('{:.0f}'.format(it1+1) + '\n' + '({:n})'.format(round(it4 * pow(10, -offset_multip_obv),2)),
                            xy=(channel_H_transform(it2 * 1.35), channel_W_transform(it3 * 1.35)), xycoords='data',
                            horizontalalignment='center', verticalalignment='center',fontsize=11)    
        
        ax_radial.text(0.98, 1.02, vars_annotate[var_id], horizontalalignment='right', verticalalignment='bottom', size = 12,
                        transform=ax_radial.transAxes)
        
        range_xlim = ax_radial.get_xlim()[1] - ax_radial.get_xlim()[0]
        range_ylim = ax_radial.get_ylim()[1] - ax_radial.get_ylim()[0]
        new_xlim = [x + y for x,y in zip(ax_radial.get_xlim(), [-range_xlim*0.3, range_xlim*0.3])]
        new_ylim = [x + y for x,y in zip(ax_radial.get_ylim(), [-range_ylim*0.3, range_ylim*0.3])]
        ax_radial.set_xlim(new_xlim); ax_radial.set_ylim(new_ylim)
        
        ax_radial.set_title(f'{vars_unit[vars_]} ' + r'$\times 10^{' + f'{offset_multip_obv}' + r'}$', pad=15)
        
        ax_radial.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_radial.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5, verticalalignment='center')
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\explore'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\explore')
            # print('Plotting results...')
            fig_vs_time.savefig(f'{vars_}_time_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            fig_radial.savefig(f'{vars_}_dist_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_plot.pdf')
            plt.close(fig_vs_time)
            plt.close(fig_radial)

def plot_coef_model(coef_curve_l, coef_rs_pred_l, save_: bool = True):
    vars_l = list(coef_curve_l.keys())
    vars_annotate = [r'$\beta_{1}$', r'$\beta_{2}$', r'$\beta_{1}$', r'$\beta_{2}$']
    vars_annotate_pred = [r'$\hat{\beta}_{1}$', r'$\hat{\beta}_{2}$', r'$\hat{\beta}_{1}$', r'$\hat{\beta}_{2}$']
    
    # identify max-min, center (9)
    get_dist = {}
    for it1, it2 in coef_curve_l.items():
        get_dist[it1] = [x - it2[-1] for x in it2]
    get_max = dict(zip(vars_l, [np.where(x >= np.max(x))[0][0] for x in list(get_dist.values())]))
    get_min = dict(zip(vars_l, [np.where(x <= np.min(x))[0][0] for x in list(get_dist.values())]))
    
    # prep radial plot
    sort_doe = [4,3,6,1,5,0,7,2]
    angle_doe = np.linspace(0,2*np.pi,9,endpoint=True)

    loc_doe = np.array([[scos(x), ssin(x)] for x in angle_doe]); loc_doe = loc_doe.T

    channel_H_trans = channel_H[sort_doe]
    channel_W_trans = channel_W[sort_doe]
    
    vars_nounit = [r'$\eta$',
                   r'$\eta$',
                   r'$\mathscr{v}^{*}$',
                   r'$\mathscr{v}^{*}$',
                   ]
    vars_unit = [r'$\left . \overline{\Delta} \eta \right |_{9}$',
                r'$\left . \overline{\Delta} \eta \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}^{*} \right |_{9}$',
                r'$\left . \overline{\Delta} \mathscr{v}^{*} \right |_{9}$'
                ]
    vars_rad = [r'$\eta$',
                r'$\eta$',
                r'$\mathscr{v}^{*}$',
                r'$\mathscr{v}^{*}$'
                ]
    
    vars_nounit = dict(zip(vars_l, vars_nounit))
    vars_unit = dict(zip(vars_l, vars_unit))
    vars_offset = dict(zip(vars_l, [math.floor(math.log10(np.max([np.abs(x) for x in coef_curve_l[x]]))) for x in vars_l]))
    vars_rad = dict(zip(vars_l, vars_rad))
    
    # plot vs time and radial dist plot
    vars_norm = dict.fromkeys(vars_l, np.zeros(shape=(2,), dtype=float))
    
    for var_id, vars_ in enumerate(coef_curve_l):
        # radial plot
        fig_radial = plt.figure(figsize=(3,3))
        ax_radial = fig_radial.add_subplot(1,1,1)
        
        loc_doe_x = np.array([channel_H_transform(x) for x in loc_doe[0]])
        loc_doe_y = np.array([channel_W_transform(x) for x in loc_doe[1]])
        
        get_dist_temp = [get_dist[vars_][x] for x in sort_doe]
        
        max_obv = np.max([np.abs(x) for x in get_dist_temp])        
        offset_multip_obv = math.floor(math.log10(max_obv))
        
        radius_obv_doe = np.array([np.abs(x) / max_obv for x in get_dist_temp])
        
        loc_obv_doe = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, radius_obv_doe)]); loc_obv_doe = loc_obv_doe.T
        loc_obv_doe_vec = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, get_dist_temp / max_obv)]); loc_obv_doe_vec = loc_obv_doe_vec.T
        
        avg_vec = np.array([np.sum(loc_obv_doe_vec[0]), np.sum(loc_obv_doe_vec[1])], dtype='float')
        avg_vec = avg_vec.astype('float') / np.linalg.norm(avg_vec)
        vars_norm[vars_] = avg_vec
        
        loc_obv_doe_x = np.array([channel_H_transform(x) for x in loc_obv_doe[0]])
        loc_obv_doe_y = np.array([channel_W_transform(x) for x in loc_obv_doe[1]])
        
        ax_radial.scatter(loc_doe_x, loc_doe_y, marker='s', **pointstyle2_dict)
        ax_radial.scatter(loc_obv_doe_x, loc_obv_doe_y, marker='s', **pointstyle1_dict)

        for it1, it2 in zip(loc_obv_doe_x, loc_obv_doe_y):
            ax_radial.plot([mean_H, it1], [mean_W, it2], color='black', linestyle='solid', linewidth=1)
        
        ax_radial.annotate('', xy=(round(channel_H_transform(avg_vec[0]), 2), round(channel_W_transform(avg_vec[1]), 2)), xytext=(channel_H_transform(0), channel_W_transform(0)),
                           xycoords='data', arrowprops=dict(arrowstyle='->', linestyle='dashed', color='red', alpha=1))
        ax_radial.annotate('< {:n}'.format(round(avg_vec[0],2)) + r'$\hat{i}$' + ' + {:n}'.format(round(avg_vec[1],2)) + r'$\hat{j}$' + ' >', xy=(0.98,0.94), xycoords='axes fraction',
                            horizontalalignment='right', verticalalignment='center',
                            bbox = dict(boxstyle=f"square,pad={0.15}", fc="None", ec="black", linewidth=0.5), fontsize=10)
        
        for it1, it2, it3, it4 in zip(sort_doe, loc_doe[0], loc_doe[1], get_dist_temp):
            ax_radial.annotate('{:.0f}'.format(it1+1) + '\n' + '({:n})'.format(round(it4 * pow(10, -offset_multip_obv),2)),
                            xy=(channel_H_transform(it2 * 1.35), channel_W_transform(it3 * 1.35)), xycoords='data',
                            horizontalalignment='center', verticalalignment='center',fontsize=11)    
        
        ax_radial.text(0.98, 1.02, vars_annotate[var_id], horizontalalignment='right', verticalalignment='bottom',
                        transform=ax_radial.transAxes)
        
        range_xlim = ax_radial.get_xlim()[1] - ax_radial.get_xlim()[0]
        range_ylim = ax_radial.get_ylim()[1] - ax_radial.get_ylim()[0]
        new_xlim = [x + y for x,y in zip(ax_radial.get_xlim(), [-range_xlim*0.3, range_xlim*0.3])]
        new_ylim = [x + y for x,y in zip(ax_radial.get_ylim(), [-range_ylim*0.3, range_ylim*0.3])]
        ax_radial.set_xlim(new_xlim); ax_radial.set_ylim(new_ylim)
        
        ax_radial.set_title(f'{vars_unit[vars_]} ' + r'$\times 10^{' + f'{offset_multip_obv}' + r'}$', pad=15)
        
        ax_radial.set_xlabel(r'$H$' + ' ' + r'$[m]$', labelpad=10)
        ax_radial.set_ylabel(r'$W$' + ' ' + r'$[m]$', labelpad=12.5, verticalalignment='center')
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\explore'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\explore')
            # print('Plotting results...')
            fig_radial.savefig(f'{vars_}_dist_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_plot.pdf')
            plt.close(fig_radial)

    get_dist = {}
    for it1, it2 in coef_rs_pred_l.items():
        get_dist[it1] = [x - it2[-1] for x in it2]
    get_max = dict(zip(vars_l, [np.where(x >= np.max(x))[0][0] for x in list(get_dist.values())]))
    get_min = dict(zip(vars_l, [np.where(x <= np.min(x))[0][0] for x in list(get_dist.values())]))

    vars_offset = dict(zip(vars_l, [math.floor(math.log10(np.max([np.abs(x) for x in coef_rs_pred_l[x]]))) for x in vars_l]))
    vars_norm = dict.fromkeys(vars_l, np.zeros(shape=(2,), dtype=float))

    for var_id, vars_ in enumerate(coef_rs_pred_l):
        # radial plot
        fig_radial = plt.figure(figsize=(3,3))
        ax_radial = fig_radial.add_subplot(1,1,1)
        
        loc_doe_x = np.array([channel_H_transform(x) for x in loc_doe[0]])
        loc_doe_y = np.array([channel_W_transform(x) for x in loc_doe[1]])
        
        get_dist_temp = [get_dist[vars_][x] for x in sort_doe]
        
        max_obv = np.max([np.abs(x) for x in get_dist_temp])        
        offset_multip_obv = math.floor(math.log10(max_obv))
        
        radius_obv_doe = np.array([np.abs(x) / max_obv for x in get_dist_temp])
        
        loc_obv_doe = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, radius_obv_doe)]); loc_obv_doe = loc_obv_doe.T
        loc_obv_doe_vec = np.array([[scos(x) * y, ssin(x) * y] \
                for x,y in zip(angle_doe, get_dist_temp / max_obv)]); loc_obv_doe_vec = loc_obv_doe_vec.T
        
        avg_vec = np.array([np.sum(loc_obv_doe_vec[0]), np.sum(loc_obv_doe_vec[1])], dtype='float')
        avg_vec = avg_vec.astype('float') / np.linalg.norm(avg_vec)
        vars_norm[vars_] = avg_vec
        
        loc_obv_doe_x = np.array([channel_H_transform(x) for x in loc_obv_doe[0]])
        loc_obv_doe_y = np.array([channel_W_transform(x) for x in loc_obv_doe[1]])
        
        ax_radial.scatter(loc_doe_x, loc_doe_y, marker='s', **pointstyle2_dict)
        ax_radial.scatter(loc_obv_doe_x, loc_obv_doe_y, marker='s', **pointstyle1_dict)

        for it1, it2 in zip(loc_obv_doe_x, loc_obv_doe_y):
            ax_radial.plot([mean_H, it1], [mean_W, it2], color='black', linestyle='solid', linewidth=1)
        
        ax_radial.annotate('', xy=(round(channel_H_transform(avg_vec[0]), 2), round(channel_W_transform(avg_vec[1]), 2)), xytext=(channel_H_transform(0), channel_W_transform(0)),
                           xycoords='data', arrowprops=dict(arrowstyle='->', linestyle='dashed', color='red', alpha=1))
        ax_radial.annotate('< {:n}'.format(round(avg_vec[0],2)) + r'$\hat{i}$' + ' + {:n}'.format(round(avg_vec[1],2)) + r'$\hat{j}$' + ' >', xy=(0.98,0.94), xycoords='axes fraction',
                            horizontalalignment='right', verticalalignment='center',
                            bbox = dict(boxstyle=f"square,pad={0.15}", fc="None", ec="black", linewidth=0.5), fontsize=10)
        
        for it1, it2, it3, it4 in zip(sort_doe, loc_doe[0], loc_doe[1], get_dist_temp):
            ax_radial.annotate('{:.0f}'.format(it1+1) + '\n' + '({:n})'.format(round(it4 * pow(10, -offset_multip_obv),2)),
                            xy=(channel_H_transform(it2 * 1.35), channel_W_transform(it3 * 1.35)), xycoords='data',
                            horizontalalignment='center', verticalalignment='center',fontsize=11)    
        
        ax_radial.text(0.98, 1.02, vars_annotate_pred[var_id], horizontalalignment='right', verticalalignment='bottom',
                        transform=ax_radial.transAxes)
        
        range_xlim = ax_radial.get_xlim()[1] - ax_radial.get_xlim()[0]
        range_ylim = ax_radial.get_ylim()[1] - ax_radial.get_ylim()[0]
        new_xlim = [x + y for x,y in zip(ax_radial.get_xlim(), [-range_xlim*0.3, range_xlim*0.3])]
        new_ylim = [x + y for x,y in zip(ax_radial.get_ylim(), [-range_ylim*0.3, range_ylim*0.3])]
        ax_radial.set_xlim(new_xlim); ax_radial.set_ylim(new_ylim)
        
        ax_radial.set_title(f'{vars_unit[vars_]} ' + r'$\times 10^{' + f'{offset_multip_obv}' + r'}$', pad=15)
        
        ax_radial.set_xlabel(r'$H$' + ' ' + r'$[m]$', labelpad=10)
        ax_radial.set_ylabel(r'$W$' + ' ' + r'$[m]$', labelpad=12.5, verticalalignment='center')
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\explore'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\explore', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\explore')
            # print('Plotting results...')
            fig_radial.savefig(f'{vars_}_model_dist_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_plot.pdf')
            plt.close(fig_radial)

# plot model
# and plot QQ plot
def plot_qq_model(df_data, save_:bool = True):
    # plot aktual reg_eta_y, reg_v_y vs reg_eta_y_model, reg_v_y_model
    # phi = 1 - beta * exp(-NuFo)
    
    vars_unit = [r'$\eta$', r'$\mathscr{v}^{*}$', r'$\overline{\mathscr{v}}$', r'$R_{t}$', r'ACH', r'$\overline{T}$', r'$h \, [\eta]$', r'$h \, [\mathscr{v}^{*}]$']
    vars_offset = [math.floor(math.log10(np.max(df_data['eta_model'].values))),
                   math.floor(math.log10(np.max(df_data['psi_model'].values))),
                   math.floor(math.log10(np.max(df_data['v_stream_model'].values))),
                   math.floor(math.log10(np.max(df_data['Rt_model'].values))),
                   math.floor(math.log10(np.max(df_data['ACH_model'].values))),
                   math.floor(math.log10(np.max(df_data['T_fluid_model'].values))),
                   math.floor(math.log10(np.max(df_data['h_eta_model'].values))),
                   math.floor(math.log10(np.max(df_data['h_psi_model'].values)))]
    
    ref_data = df_data[df_data['case'] == 9]

    for var_id, vars_ in enumerate(['eta', 'psi', 'v_stream', 'Rt', 'ACH', 'T_fluid', 'h_eta', 'h_psi']):
        fig_qq_model = plt.figure(figsize=(3.75,3))
        ax_qq_model = fig_qq_model.add_subplot(111)
        
        res_beta = df_data[vars_].values - df_data[f'{vars_}_model'].values
        std_beta = np.std(res_beta)
        ksval_beta, kspval_beta = stats.ks_2samp(res_beta, stats.norm.rvs(size=res_beta.shape[0], loc=0, scale=std_beta))
        try:
            dwval_beta = smstats.stattools.durbin_watson(res_beta)
        except:
            dwval_beta = 1
            std_beta = 1e-5
        
        sm.graphics.qqplot(res_beta, stats.norm, loc=0, scale=std_beta, line=None, ax=ax_qq_model)
        ax_qq_model.plot(ax_qq_model.get_xlim(), ax_qq_model.get_ylim(), linestyle='dashed')
        
        try:
            unit_std_beta = math.floor(math.log10(std_beta))
        except:
            unit_std_beta = -5
        
        legend_box = 'KS = {:n} ({:n})\nDW = {:n}'.format(round(ksval_beta, 2), round(kspval_beta,2), round(dwval_beta, 2)) 
        ax_qq_model.annotate(legend_box, xy=(0.95, 0.2), xycoords='axes fraction',
                    horizontalalignment='right', verticalalignment='top',
                    bbox = dict(boxstyle=f"square,pad={0.35}", fc="None", ec="black"), fontsize=12)
        
        a = ax_qq_model.get_xticks()
        
        x_ticks_offset = math.floor(math.log10(np.max([np.abs(float(x)) for x in ax_qq_model.get_xticks()])))
        y_ticks_offset = math.floor(math.log10(np.max([np.abs(float(x)) for x in ax_qq_model.get_yticks()])))
        ax_qq_model.set_xticklabels(['{:n}'.format(round(x * pow(10, -x_ticks_offset), 2)) for x in ax_qq_model.get_xticks()])
        ax_qq_model.set_yticklabels(['{:n}'.format(round(x * pow(10, -y_ticks_offset), 2)) for x in ax_qq_model.get_yticks()])
        
        ax_qq_model.text(0.02, 1.1, r'$\times 10^{' + f'{x_ticks_offset}' + r'}$',
                         horizontalalignment='left', verticalalignment='top', size = y_offset_fontsize,
                         transform=ax_qq_model.transAxes)
        ax_qq_model.text(1.1, -0.2, r'$\times 10^{' + f'{y_ticks_offset}' + r'}$',
                         horizontalalignment='right', verticalalignment='center', size = y_offset_fontsize,
                         transform=ax_qq_model.transAxes)
        
        # ax_qq_model.set_xlabel('Kuantil teoritis ~ ' + 'N(0, {:n}'.format(round(std_beta * \
        #         pow(10, -unit_std_beta))) + r'$\times 10^{' + f'{unit_std_beta}' + r'})$',
        #         horizontalalignment='center', labelpad=15)
        ax_qq_model.set_xlabel('Kuantil teoritis', horizontalalignment='center', labelpad=10)
        ax_qq_model.set_ylabel('Kuantil sampel', verticalalignment='center', labelpad=12.5)        
        ax_qq_model.set_title(vars_unit[var_id], fontsize=16, pad=15)

        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\analysis'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\analysis')
            # print('Plotting results...')
            fig_qq_model.savefig(f'{vars_}_qq_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_model_qq_plot.pdf')
            plt.close(fig_qq_model)

# plot ccd
def plot_model_3d_response(df_data, coef_rs_l, save_:bool = True):
    # 3d rs coefs
    radius_doe, angle_doe = np.meshgrid(np.linspace(0,1,10,endpoint=True), np.linspace(0,2*np.pi,36,endpoint=True))
    angle_doe_connect = np.linspace(0,2*np.pi,36,endpoint=True)
    H_doe = np.array([channel_H_transform(x) for x in radius_doe * np.cos(angle_doe)])
    W_doe = np.array([channel_W_transform(x) for x in radius_doe * np.sin(angle_doe)])
    H_doe_connect = np.array([channel_H_transform(x) for x in np.cos(angle_doe_connect)])
    W_doe_connect = np.array([channel_W_transform(x) for x in np.sin(angle_doe_connect)])
    
    vars_unit = [r'$\eta$', r'$\mathscr{v}^{*}$']

    max_time = 3600
    
    for var_id, vars_ in enumerate(['eta', 'v']):
        b1_coef_l = coef_rs_l[f'{vars_}_b1']
        b2_coef_l = coef_rs_l[f'{vars_}_b2']
                
        pred_doe_beta = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, x, y, max_time) for \
            x,y in zip(H_doe.ravel(), W_doe.ravel())])
        pred_doe_beta_init = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, x, y, 0) for \
            x,y in zip(H_doe.ravel(), W_doe.ravel())])
        pred_doe_beta_connect = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, x, y, max_time) for \
            x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])
        pred_doe_beta_init_connect = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, x, y, 0) for \
            x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])

        vars_offset = math.floor(math.log10(np.max([np.abs(x) for x in df_data[f'{vars_trans[vars_]}_model'].values])))
        offset_multip_beta = vars_offset 
        
        pred_doe_beta = pred_doe_beta.reshape(*radius_doe.shape) * pow(10, -offset_multip_beta)
        pred_doe_beta_init = pred_doe_beta_init.reshape(*radius_doe.shape) * pow(10, -offset_multip_beta)
        pred_doe_beta_connect = pred_doe_beta_connect * pow(10, -offset_multip_beta)
        pred_doe_beta_init_connect = pred_doe_beta_init_connect * pow(10, -offset_multip_beta)
        
        # 3d rs
        fig_rs_beta = plt.figure(figsize=(4,4), dpi=300)
        ax_rs_beta = fig_rs_beta.add_subplot(111, projection='3d')
        ax_surf_beta = ax_rs_beta.plot_surface(H_doe, W_doe, pred_doe_beta, cmap=cm.coolwarm)
        
        range_beta = np.linspace(np.min([pred_doe_beta.min(), pred_doe_beta_init.min()]), np.max([pred_doe_beta.max(), pred_doe_beta_init.max()]), 4, endpoint=True)
        
        cbar_beta = fig_rs_beta.colorbar(ax_surf_beta, shrink=0.25, aspect=20, location='right',
                                        pad=0.025, orientation='vertical')
        cbar_beta.ax.yaxis.set_ticks_position('right'); cbar_beta.ax.yaxis.labelpad = 10
        cbar_beta.set_ticks([x // 0.01 / 100 for x in range_beta])
        cbar_beta.set_ticklabels(['{:n}'.format(round(x // 0.01 / 100, 2)) for x in range_beta])
        # [x.set_fontsize(12) for x in cbar_beta.ax.get_yticklabels()]
        ax_rs_beta.plot_wireframe(H_doe, W_doe, pred_doe_beta, alpha=0.5, linewidth=0.5, edgecolors='black')
        ax_rs_beta.plot_wireframe(H_doe, W_doe, pred_doe_beta_init, alpha=0.5, linewidth=0.5, edgecolors='black')
        
        for line_x, line_y, line_z, line_z0 in zip(H_doe_connect, W_doe_connect, pred_doe_beta_connect, pred_doe_beta_init_connect):
            ax_rs_beta.plot([line_x, line_x], [line_y, line_y], [line_z, line_z0], alpha=0.5, linewidth=0.5, color='black')
        
        # ax_rs_beta.ticklabel_format(style='sci', scilimits=(-2,0), useMathText=True, useOffset=False, axis='z')
        ax_rs_beta.set_proj_type('ortho')
        ax_rs_beta.view_init(elev=45, azim=45, roll=0)
        # ax_rs_beta.zaxis.set_rotate_label(False)
        ax_rs_beta.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=8)
        ax_rs_beta.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=8)
        ax_rs_beta.set_zlabel(vars_unit[var_id], labelpad=10)
        ax_rs_beta.text2D(0, 0.8, r'$\times 10^{' + f'{offset_multip_beta}' + r'}$',
                        size=y_offset_fontsize, transform=ax_rs_beta.transAxes)
        ax_rs_beta.xaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.yaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.zaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.xaxis._axinfo['grid']['color'] = (0,0,0,1)
        ax_rs_beta.yaxis._axinfo['grid']['color'] = (0,0,0,1)
        ax_rs_beta.zaxis._axinfo['grid']['color'] = (0,0,0,1)
        # ax_rs_eta.set_title('Koefisien kurva ' + r'$\overline{\mathscr{v}}$', pad=15)
        
        ax_rs_beta.set_zticks([round(x, 2) for x in range_beta])
        
        fig_rs_beta.subplots_adjust(left=0.3)
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\analysis'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\analysis')
            # print('Plotting results...')
            fig_rs_beta.savefig(f'{vars_}_3d_rs.pdf', transparent=True, format='pdf')
            print('Saved to x_3d_rs.pdf')
            plt.close(fig_rs_beta)
    
    sort_doe = [4,3,6,1,5,0,7,2]
    angle_doe = np.linspace(0,2*np.pi,9,endpoint=True)

    loc_doe = np.array([[np.cos(x), np.sin(x)] for x in angle_doe]); loc_doe = loc_doe.T

    channel_H_trans = channel_H[sort_doe]
    channel_W_trans = channel_W[sort_doe]
    
    range_H = np.linspace(np.min(channel_H) - 0.05 * (np.max(channel_H) - np.min(channel_H)),
                          np.max(channel_H) + 0.05 * (np.max(channel_H) - np.min(channel_H)), 50, endpoint=True)
    range_W = np.linspace(np.min(channel_W) - 0.05 * (np.max(channel_W) - np.min(channel_W)),
                          np.max(channel_W) + 0.05 * (np.max(channel_W) - np.min(channel_W)), 50, endpoint=True)
    
    H_doe, W_doe = np.meshgrid(range_H, range_W)
    pred_l = list(zip(H_doe.ravel(), W_doe.ravel()))
    
    for var_id, vars_ in enumerate(['eta', 'v']):
        b1_coef_l = coef_rs_l[f'{vars_}_b1']
        b2_coef_l = coef_rs_l[f'{vars_}_b2']
        
        vars_offset = math.floor(math.log10(np.max([np.abs(x) for x in df_data[f'{vars_trans[vars_]}_model'].values])))
        
        # 2d dens
        fig_dens_beta = plt.figure(figsize=(3,3))
        fig_dens_beta_init = plt.figure(figsize=(3,3))
        ax_dens_beta = fig_dens_beta.add_subplot(1,1,1)
        ax_dens_beta_init = fig_dens_beta_init.add_subplot(1,1,1)
        
        dens_data_beta = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, *x, max_time) for x in pred_l])
        dens_data_beta_init = np.array([coef_func[vars_](b1_coef_l, b2_coef_l, *x, max_time/2) for x in pred_l])
        dens_data_beta = dens_data_beta * pow(10, -vars_offset)
        dens_data_beta_init = dens_data_beta_init * pow(10, -vars_offset)
        dens_data_beta = dens_data_beta.reshape(H_doe.shape)
        dens_data_beta_init = dens_data_beta_init.reshape(H_doe.shape)
        
        lev_ = np.array([np.quantile(dens_data_beta, x) for x in np.linspace(0, 1, 8, endpoint=True)])
        lev_init = np.array([np.quantile(dens_data_beta_init, x) for x in np.linspace(0, 1, 8, endpoint=True)])
        dens_fig_beta = ax_dens_beta.contour(H_doe, W_doe, dens_data_beta, levels=lev_, colors='black')
        dens_fig_beta_init = ax_dens_beta_init.contour(H_doe, W_doe, dens_data_beta_init, levels=lev_init, colors='black')
        
        loc_doe_0trans = [channel_H_transform(x) for x in loc_doe[0]]
        loc_doe_1trans = [channel_W_transform(x) for x in loc_doe[1]]
        
        ax_dens_beta.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_dens_beta.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5)
        ax_dens_beta.clabel(dens_fig_beta, dens_fig_beta.levels, inline=True, fmt=fmt, fontsize=12)
        ax_dens_beta.scatter(loc_doe_0trans, loc_doe_1trans, marker='s', **pointstyle2_dict)
        ax_dens_beta.scatter([channel_H_transform(0)], [channel_W_transform(0)], marker='s', **pointstyle2_dict)
        ax_dens_beta.text(1, 1.1, f'{max_time}s', horizontalalignment='right', verticalalignment='top', size=12,
                        transform=ax_dens_beta.transAxes)
        ax_dens_beta.set_title(vars_unit[var_id] + ' ' + r'$\times 10^{' + f'{vars_offset}' + r'}$', pad=15)

        ax_dens_beta_init.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_dens_beta_init.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5)
        ax_dens_beta_init.clabel(dens_fig_beta_init, dens_fig_beta_init.levels, inline=True, fmt=fmt, fontsize=12)
        ax_dens_beta_init.scatter(loc_doe_0trans, loc_doe_1trans, marker='s', **pointstyle2_dict)
        ax_dens_beta_init.scatter([channel_H_transform(0)], [channel_W_transform(0)], marker='s', **pointstyle2_dict)
        ax_dens_beta_init.text(1, 1.1, f'{int(round(max_time/2, 0))}s', horizontalalignment='right', verticalalignment='top', size=12,
                        transform=ax_dens_beta_init.transAxes)
        ax_dens_beta_init.set_title(vars_unit[var_id] + ' ' + r'$\times 10^{' + f'{vars_offset}' + r'}$', pad=15)
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\analysis'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\analysis')
            # print('Plotting results...')
            fig_dens_beta.savefig(f'{vars_}_dens_rs_end.pdf', transparent=True, bbox_inches='tight', format='pdf')
            fig_dens_beta_init.savefig(f'{vars_}_dens_rs_init.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_dens_rs.pdf')
            plt.close(fig_dens_beta)
            plt.close(fig_dens_beta_init)

def plot_model_3d_response_specifics(df_data, coef_rs_l, save_:bool = True):
    # 3d rs coefs
    radius_doe, angle_doe = np.meshgrid(np.linspace(0,1,10,endpoint=True), np.linspace(0,2*np.pi,36,endpoint=True))
    angle_doe_connect = np.linspace(0,2*np.pi,36,endpoint=True)
    H_doe = np.array([channel_H_transform(x) for x in radius_doe * np.cos(angle_doe)])
    W_doe = np.array([channel_W_transform(x) for x in radius_doe * np.sin(angle_doe)])
    H_doe_connect = np.array([channel_H_transform(x) for x in np.cos(angle_doe_connect)])
    W_doe_connect = np.array([channel_W_transform(x) for x in np.sin(angle_doe_connect)])
    
    vars_l = ['T_fluid', 'h_eta', 'h_psi', 'Nu_eta', 'Nu_psi', 'q', 'v_stream', 'ACH', 'Rt', 'v_dn']
    vars_nounit = [r'$(\overline{T} - T_{0})$', r'$h \, [\eta]$', r'$h \, [\mathscr{v}^{*}]$',
                   r'$Nu \, [\eta]$', r'$Nu \, [\mathscr{v}^{*}]$',
                   r'$\dot{Q}_{eff}$', r'$\left . \Delta \, \overline{\mathscr{v}}(t) \right |_{0}^{t}$',
                   r'$\mathrm{ACH}$', r'$R_{t}$', r'$\mathscr{v}_{DN, \, eff}$']
    vars_unit = ['[K]', r'$[\mathrm{W}\mathrm{m}^{-2}\mathrm{K}^{-1}]$', r'$[\mathrm{W}\mathrm{m}^{-2}\mathrm{K}^{-1}]$', '', '', '[W]', r'$[\mathrm{m}\mathrm{s}^{-1}]$', '', '', r'$[\mathrm{m}\mathrm{s}^{-1}]$']
    vars_init = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    vars_offset_l = []
    
    max_time = 3600
    
    for var_id, vars_ in enumerate(vars_l):
        vars_offset = math.floor(math.log10(np.max([np.abs(x) for x in df_data[f'{vars_}_model'].values])))
        
        if vars_ in ['Rt']:
            pred_doe_beta = np.array([coef_func[vars_](coef_rs_l, x, y, max_time) for \
                x,y in zip(H_doe.ravel(), W_doe.ravel())])
            pred_doe_beta_init = np.array([1 for \
                x,y in zip(H_doe.ravel(), W_doe.ravel())])
            pred_doe_beta_connect = np.array([coef_func[vars_](coef_rs_l, x, y, max_time) for \
                x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])
            pred_doe_beta_init_connect = np.array([1 for \
                x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])
        else:
            pred_doe_beta = np.array([coef_func[vars_](coef_rs_l, x, y, max_time) - vars_init[var_id] for \
                x,y in zip(H_doe.ravel(), W_doe.ravel())])
            pred_doe_beta_init = np.array([coef_func[vars_](coef_rs_l, x, y, 0) - vars_init[var_id] for \
                x,y in zip(H_doe.ravel(), W_doe.ravel())])
            pred_doe_beta_connect = np.array([coef_func[vars_](coef_rs_l, x, y, max_time) - vars_init[var_id] for \
                x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])
            pred_doe_beta_init_connect = np.array([coef_func[vars_](coef_rs_l, x, y, 0) - vars_init[var_id] for \
                x,y in zip(H_doe_connect.ravel(), W_doe_connect.ravel())])

        if vars_ in ['v_stream']:
            pred_doe_beta = pred_doe_beta - pred_doe_beta_init
            pred_doe_beta_init = pred_doe_beta_init - pred_doe_beta_init
            pred_doe_beta_connect = pred_doe_beta_connect - pred_doe_beta_init_connect
            pred_doe_beta_init_connect = pred_doe_beta_init_connect - pred_doe_beta_init_connect

        vars_offset_beta = math.floor(math.log10(np.max([np.abs(x) for x in pred_doe_beta_connect])))
        vars_offset_beta_init = 0
        # vars_offset_beta_init = math.floor(math.log10(np.max([np.abs(x) for x in pred_doe_beta_init_connect])))
        vars_offset = np.min([vars_offset_beta, vars_offset_beta_init])
        vars_offset_l.append(float(vars_offset)) 
        offset_multip_beta = float(vars_offset)
        
        pred_doe_beta = pred_doe_beta.reshape(*radius_doe.shape) * pow(10, -offset_multip_beta)
        pred_doe_beta_init = pred_doe_beta_init.reshape(*radius_doe.shape) * pow(10, -offset_multip_beta)
        pred_doe_beta_connect = pred_doe_beta_connect * pow(10, -offset_multip_beta)
        pred_doe_beta_init_connect = pred_doe_beta_init_connect * pow(10, -offset_multip_beta)
        
        # 3d rs
        fig_rs_beta = plt.figure(figsize=(4,4), dpi=300)
        ax_rs_beta = fig_rs_beta.add_subplot(111, projection='3d')
        ax_surf_beta = ax_rs_beta.plot_surface(H_doe, W_doe, pred_doe_beta, cmap=cm.coolwarm)
        
        range_beta = np.linspace(np.min([pred_doe_beta.min(), pred_doe_beta_init.min()]), np.max([pred_doe_beta.max(), pred_doe_beta_init.max()]), 4, endpoint=True)
        
        cbar_beta = fig_rs_beta.colorbar(ax_surf_beta, shrink=0.25, aspect=20, location='right',
                                        pad=0.025, orientation='vertical')
        cbar_beta.ax.yaxis.set_ticks_position('right'); cbar_beta.ax.yaxis.labelpad = 10
        cbar_beta.set_ticks([x // 0.01 / 100 for x in range_beta])
        cbar_beta.set_ticklabels(['{:n}'.format(round(x // 0.01 / 100, 2)) for x in range_beta])
        # [x.set_fontsize(12) for x in cbar_beta.ax.get_yticklabels()]
        ax_rs_beta.plot_wireframe(H_doe, W_doe, pred_doe_beta, alpha=0.5, linewidth=0.5, edgecolors='black')
        ax_rs_beta.plot_wireframe(H_doe, W_doe, pred_doe_beta_init, alpha=0.5, linewidth=0.5, edgecolors='black')
        
        for line_x, line_y, line_z, line_z0 in zip(H_doe_connect, W_doe_connect, pred_doe_beta_connect, pred_doe_beta_init_connect):
            ax_rs_beta.plot([line_x, line_x], [line_y, line_y], [line_z, line_z0], alpha=0.5, linewidth=0.5, color='black')
        
        # ax_rs_beta.ticklabel_format(style='sci', scilimits=(-2,0), useMathText=True, useOffset=False, axis='z')
        ax_rs_beta.set_proj_type('ortho')
        ax_rs_beta.view_init(elev=45, azim=45, roll=0)
        # ax_rs_beta.zaxis.set_rotate_label(False)
        ax_rs_beta.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=8)
        ax_rs_beta.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=8)
        ax_rs_beta.set_zlabel(vars_nounit[var_id] + ' ' + vars_unit[var_id], labelpad=10)
        ax_rs_beta.text2D(0, 0.8, r'$\times 10^{' + f'{int(offset_multip_beta)}' + r'}$',
                        size=y_offset_fontsize, transform=ax_rs_beta.transAxes)
        ax_rs_beta.xaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.yaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.zaxis.set_pane_color((1,1,1,0))
        ax_rs_beta.xaxis._axinfo['grid']['color'] = (0,0,0,1)
        ax_rs_beta.yaxis._axinfo['grid']['color'] = (0,0,0,1)
        ax_rs_beta.zaxis._axinfo['grid']['color'] = (0,0,0,1)
        # ax_rs_eta.set_title('Koefisien kurva ' + r'$\overline{\mathscr{v}}$', pad=15)
        
        ax_rs_beta.set_zticks([round(x, 2) for x in range_beta])
        
        fig_rs_beta.subplots_adjust(left=0.3)
        
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\analysis'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\analysis')
            # print('Plotting results...')
            fig_rs_beta.savefig(f'{vars_}_3d_rs.pdf', transparent=True, format='pdf')
            print('Saved to x_3d_rs.pdf')
            plt.close(fig_rs_beta)
    
    sort_doe = [4,3,6,1,5,0,7,2]
    angle_doe = np.linspace(0,2*np.pi,9,endpoint=True)

    loc_doe = np.array([[np.cos(x), np.sin(x)] for x in angle_doe]); loc_doe = loc_doe.T

    channel_H_trans = channel_H[sort_doe]
    channel_W_trans = channel_W[sort_doe]
    
    range_H = np.linspace(np.min(channel_H) - 0.05 * (np.max(channel_H) - np.min(channel_H)),
                          np.max(channel_H) + 0.05 * (np.max(channel_H) - np.min(channel_H)), 50, endpoint=True)
    range_W = np.linspace(np.min(channel_W) - 0.05 * (np.max(channel_W) - np.min(channel_W)),
                          np.max(channel_W) + 0.05 * (np.max(channel_W) - np.min(channel_W)), 50, endpoint=True)
    
    H_doe, W_doe = np.meshgrid(range_H, range_W)
    pred_l = list(zip(H_doe.ravel(), W_doe.ravel()))

    for var_id, vars_ in enumerate(vars_l):
        # 2d dens
        fig_dens_beta = plt.figure(figsize=(3,3))
        fig_dens_beta_init = plt.figure(figsize=(3,3))
        ax_dens_beta = fig_dens_beta.add_subplot(1,1,1)
        ax_dens_beta_init = fig_dens_beta_init.add_subplot(1,1,1)
    
        dens_data_beta = np.array([coef_func[vars_](coef_rs_l, *x, max_time) - vars_init[var_id] for x in pred_l])
        dens_data_beta_init = np.array([coef_func[vars_](coef_rs_l, *x, max_time/2) - vars_init[var_id] for x in pred_l])
        
        if vars_ in ['v_stream']:
            dens_data_beta_zero = np.array([coef_func[vars_](coef_rs_l, *x, 0) - vars_init[var_id] for x in pred_l])
            dens_data_beta = dens_data_beta - dens_data_beta_zero
            dens_data_beta_init = dens_data_beta_init - dens_data_beta_zero

        vars_offset = vars_offset_l[var_id]
        
        dens_data_beta = dens_data_beta * pow(10, -vars_offset)
        dens_data_beta_init = dens_data_beta_init * pow(10, -vars_offset)
        dens_data_beta = dens_data_beta.reshape(H_doe.shape)
        dens_data_beta_init = dens_data_beta_init.reshape(H_doe.shape)
    
        lev_ = np.array([np.quantile(dens_data_beta, x) for x in np.linspace(0, 1, 8, endpoint=True)])
        lev_init = np.array([np.quantile(dens_data_beta_init, x) for x in np.linspace(0, 1, 8, endpoint=True)])
        dens_fig_beta = ax_dens_beta.contour(H_doe, W_doe, dens_data_beta, levels=lev_, colors='black')
        dens_fig_beta_init = ax_dens_beta_init.contour(H_doe, W_doe, dens_data_beta_init, levels=lev_init, colors='black')
            
        loc_doe_0trans = [channel_H_transform(x) for x in loc_doe[0]]
        loc_doe_1trans = [channel_W_transform(x) for x in loc_doe[1]]
        
        ax_dens_beta.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_dens_beta.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5)
        ax_dens_beta.clabel(dens_fig_beta, dens_fig_beta.levels, inline=True, fmt=fmt, fontsize=12)
        ax_dens_beta.scatter(loc_doe_0trans, loc_doe_1trans, marker='s', **pointstyle2_dict)
        ax_dens_beta.scatter([channel_H_transform(0)], [channel_W_transform(0)], marker='s', **pointstyle2_dict)
        ax_dens_beta.text(1, 1.1, f'{max_time}s', horizontalalignment='right', verticalalignment='top', size=12,
                        transform=ax_dens_beta.transAxes)
        ax_dens_beta.set_title(vars_nounit[var_id] + ' ' + r'$\times 10^{' + f'{int(vars_offset)}' + r'}$', pad=15)
        
        ax_dens_beta_init.set_xlabel(r'$H$' + ' ' + '[m]', labelpad=10)
        ax_dens_beta_init.set_ylabel(r'$W$' + ' ' + '[m]', labelpad=12.5)
        ax_dens_beta_init.clabel(dens_fig_beta_init, dens_fig_beta_init.levels, inline=True, fmt=fmt, fontsize=12)
        ax_dens_beta_init.scatter(loc_doe_0trans, loc_doe_1trans, marker='s', **pointstyle2_dict)
        ax_dens_beta_init.scatter([channel_H_transform(0)], [channel_W_transform(0)], marker='s', **pointstyle2_dict)
        ax_dens_beta_init.text(1, 1.1, f'{int(round(max_time/2, 0))}s', horizontalalignment='right', verticalalignment='top', size=12,
                        transform=ax_dens_beta_init.transAxes)
        ax_dens_beta_init.set_title(vars_nounit[var_id] + ' ' + r'$\times 10^{' + f'{int(vars_offset)}' + r'}$', pad=15)
    
        if save_ is True:
            if not os.path.exists(f'{caseloc}\\plot\\analysis'):
                try:
                    original_umask = os.umask(0)
                    os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
                finally:
                    os.umask(original_umask)
            os.chdir(f'{caseloc}\\plot\\analysis')
            # print('Plotting results...')
            fig_dens_beta.savefig(f'{vars_}_dens_rs_end.pdf', transparent=True, bbox_inches='tight', format='pdf')
            fig_dens_beta_init.savefig(f'{vars_}_dens_rs_init.pdf', transparent=True, bbox_inches='tight', format='pdf')
            print('Saved to x_dens_rs.pdf')
            plt.close(fig_dens_beta)
            plt.close(fig_dens_beta_init)

# plot q model
def plot_q_model(df_data, save_:bool = True):
    fig_q_model = plt.figure(figsize=(3.75,3))
    ax_q_model = fig_q_model.add_subplot(111)
    q_model_l = []
    for case_name, df_case in df_data.groupby('case'):
        ax_q_model.scatter([case_name], [df_case['q_model'].values[0]], edgecolor='black', facecolor='None')
        q_model_l.append(df_case['q_model'].values[0])
        
    ax_q_model2 = ax_q_model.twinx()
    ax_q_model2.set_yticks([np.max(q_model_l)])
    ax_q_model2.set_ylim(ax_q_model.get_ylim())
    
    ax_q_model.set_title(r'$\dot{q}_{eff}$', fontsize=14, pad=15)
    ax_q_model.set_xlabel('Kasus', labelpad=10, horizontalalignment='center')
    ax_q_model.set_ylabel(r'$\dot{q}_{eff}$', labelpad=25, verticalalignment='center')

    if save_ is True:
        if not os.path.exists(f'{caseloc}\\plot\\analysis'):
            try:
                original_umask = os.umask(0)
                os.makedirs(f'{caseloc}\\plot\\analysis', 0o777)
            finally:
                os.umask(original_umask)
        os.chdir(f'{caseloc}\\plot\\analysis')
        # print('Plotting results...')
        fig_q_model.savefig('q_model_plot.pdf', transparent=True, bbox_inches='tight', format='pdf')
        print('Saved to x_plot.pdf')
        plt.close(fig_q_model)    

# plot_vs_time(df_)
# plot_error(df_)
# plot_compound(df_)
plot_compound_model(df_)
plot_coef_model(coef_curve_l, coef_rs_pred_l)
# plot_qq_model(df_)
# plot_model_3d_response(df_, coef_rs_l)
# plot_model_3d_response_specifics(df_, coef_rs_l)

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16324\3611948719.py:842: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])


Saved to x_plot.pdf


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16324\3611948719.py:842: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])


Saved to x_plot.pdf


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16324\3611948719.py:842: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])


...

Saved to x_plot.pdf


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16324\3611948719.py:842: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])


Saved to x_plot.pdf


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16324\3611948719.py:842: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax_vs_time.set_yticklabels(['{:n}'.format(round(float(x), 2)) for x in ax_vs_time.get_yticks()])


Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf
Saved to x_plot.pdf


In [10]:
# calc surogate and surface response corr
# import numpy as np

# respons_data = {'psi': np.array([-0.99, 0.11]), 'eta': np.array([-0.99, 0.16]),
#               'v_stream': np.array([1,0]), 'ach': np.array([0.87, 0.5]), 'rt': np.array([-0.88, -0.47]),
#               'T_fluid': np.array([-0.99,0.17]), 'h': np.array([-0.99, -0.12])}

# respons_surogat = {'psi': np.array([-1, 0.05]), 'eta': np.array([-0.99, 0.12])}

# respons_rs = {'psi': np.array([-0.79, 0.61]), 'eta': np.array([-1,0]),
#               'v_stream': np.array([0.99, -0.14]), 'ach': np.array([0.87,0.5]), 'rt': np.array([-0.88, -0.47]),
#               'T_fluid': np.array([-0.97, 0.25]), 'h': np.array([0.91, 0.42])}

# b1_surogat = {'psi': np.array([-1, 0.05]), 'eta': np.array([-1, -0.09])}
# b2_surogat = {'psi': np.array([0.99, -0.11]), 'eta': np.array([0.93, 0.36])}

# b1_rs = {'psi': np.array([-1, 0]), 'eta': np.array([-1, -0.03])}
# b2_rs = {'psi': np.array([1, -0.05]), 'eta': np.array([0.77, 0.64])}

# print('respons surogat')
# [print(f'{it1}: {round( np.dot(respons_data[it1], respons_surogat[it1]) / ( np.linalg.norm(respons_data[it1]) * np.linalg.norm(respons_surogat[it1]) ) , 2)}') for it1 in list(respons_surogat.keys())]
# print('\nrespons rs')
# [print(f'{it1}: {round( np.dot(respons_data[it1], respons_rs[it1]) / ( np.linalg.norm(respons_data[it1]) * np.linalg.norm(respons_rs[it1]) ) , 2)}') for it1 in list(respons_data.keys())]
# print('\nb1')
# [print(f'{it1}: {round( np.dot(b1_surogat[it1], b1_rs[it1]) / ( np.linalg.norm(b1_surogat[it1]) * np.linalg.norm(b1_rs[it1]) ) , 2)}') for it1 in list(b1_surogat.keys())]
# print('\nb2')
# [print(f'{it1}: {round( np.dot(b2_surogat[it1], b2_rs[it1]) / ( np.linalg.norm(b2_surogat[it1]) * np.linalg.norm(b2_rs[it1]) ) , 2)}') for it1 in list(b2_surogat.keys())]

# # clockwise from 1 -> 9
# respons_data = {'psi': np.array([]),
#                 'eta': np.array([]),
#                 'v_stream': np.array([]),
#                 'ach': np.array([]),
#                 'rt': np.array([]),
#                 'T_fluid': np.array([]),
#                 'h': np.array([])}

# respons_surogat = {'psi': np.array([]),
#                    'eta': np.array([]),
#                    'v_stream': np.array([]),
#                    'ach': np.array([]),
#                    'rt': np.array([]),
#                    'T_fluid': np.array([]),
#                    'h': np.array([])}

# b1_surogat = {'psi': np.array([]),
#               'eta': np.array([])}

# b2_surogat = {'psi': np.array([]),
#               'eta': np.array([])}

# b1_rs = {'psi': np.array([2.06, 2.88, 2.07, 0, -2.06, -2.88, -2.07, 0, 0]),
#          'eta': np.array([5.4, 8, 6.43, 0.45, -5.95, -8, -4.42, 1.18, 5.4])}

# b2_rs = {'psi': np.array([-1.25, -1.84, -1.39, -0.1, 1.25, 1.84, 1.39, 0.1, 0]),
#          'eta': np.array([-1.05, -0.98, -0.37, 0.82, 1.53, 0.98, ])}

# offset_var = {'b1_psi': -5, 'b2_psi': -5,
#               'b1_eta': -6, 'b2_eta': -3,
#               'psi': , 'eta': ,
#               'v_stream': , 'ach': , 'rt': ,
#               'T_fluid': , 'h'}

# def calc_f(arr1, arr2, df_galat):
#     return

# []
